In [6]:
import re
string = "My mom loves a tiger which is the national animal of INDIA!"
pattern = "Tiger"
result = re.match(pattern,string)
print(result.group(0))

Tiger


In [7]:
mo = re.search(pattern, string)
print(mo.group(0))

Tiger


In [10]:
pattern2 = "national"
mo1 = re.sub(pattern,'love',string)
print(mo1)

love is the national animal of INDIA!


In [2]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =[' - ', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']

        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
        print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        for word in token_pairs:
            print(word)
        token_pairs_freq = pd.Series(token_pairs).value_counts()    
        print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''trevarsa tainka kabutara, mora aura titara jaisi vibhinna jatiyom ke pakshiyom ke karana pakshi - avalokana ke lie prasiddha hai | guru shikhara maunta abu ki sabase unchi choti hai jaham se pure shahara ka bahuta achchha naja़ara dikhata hai| parvata ke shikhara para sthita eka chhota-sa shaiva punya-sthala aura dattatreya ka mandira bhramana ke do mahatvapurna sthana haim| rajasthana ke thara marusthala ke asali jadu ka anubhava pane ka sabase achchha tarika hai marusthala sapha़ari | maunta abu mem rajasthali, rajasthana shasakiya hastashilpa emporiyama aura khadi bhandara kharidari ke mahatvapurna sthana haim | apraila se juna aura aktubara se navambara ka samaya isa shahara ke bhramana ke lie sabase achchha hai | sahasika karyom ki talasha mem nikale paryatakom ke lie thara marusthala ke andaruni hissom mem ghumane, shaharom aura aitihasika khandaharom ko dekhane ke lie unta sapha़ari sarvottama vikalpa hai | rajasthana ke chhote- chhote gamvom mem ghumane ke anokhe anubhava ke lie unta sapha़ari eka romanchaka aura jokhima -bhara sadhana hai | thara marusthala ke khule bhubhaga para ghumane lie jipa sapha़ari eka adbhuta sadhana hai aura rajasthana ke khubasurata, shanta aura ranga-birange ilake ko dekhane ka anokha avasara pradana karati hai | ananda aura romancha ke mishrana ke satha rajasthana ke thara marusthala mem jipa sapha़ari vaham ke kuchha shanadara gantavyom ko samavishta karati hai| thara marusthala ke bhavya retile tile rajasthana ki anokhi dena haim jo ki vaham ki ranga-birangi sanskriti,vanya - jivana, aitihasika smarakom, kilom, udyanom aura jilom se alaga haim | insani hunara aura acharana se anachhue rajasthana ke ye retile tile prakriti ka eka uttama upahara haim jo registana ka eka jadui naja़ara pesha karate haim | nile asamana ke niche hava se akara mem dhale retile tile duniya -bhara se bada़i sankhya mem paryatakom ko akarshita karate haim aura rajasthana ki koi bhi saira ina retile tilom ki saira ke bina adhuri hai | retile tilom dvara bani laharem photographa़rom ke lie pha़oto khinchane ke lie uttama sthana rachati haim | rajasthana ke marusthala ki saira karane ka sarvottama tarika unta sapha़ari ke madhyama se hai, jo registani paryatana ka sabase lokapriya sadhana hai | rajasthana ke andaruni hisse sirpha़ isi sadhana ke jarie dekhe ja sakate haim jise prayah ‘registana ka jahaja’ kaha jata hai | rajasthana mem unta sapha़ari solahavim shatabdi se prachalana mem hai jise taba mukhya rupa se samana dhone ke lie istemala kiya jata tha, aura aba yaha manoranjana ka mahatvapurna sadhana hai | rajasthana ke thara marusthala mem chalane vale unta sapha़ari ke aja ke rupa mem prachina unta- karavam ka jadu dekha ja sakata hai | rajasthana - bhramana ka eka mahatvapurna hissa, unta sapha़ari paramparika aura ranga-birange rajasthana ki andaruni janki dikhata hai |
jaisalamera, jodhapura aura bikanera retile tilom ke lie jane jate haim para inamem se sabase lokapriya jaisalamera hai | adhikatara reta ke tile jaisalamera ke asapasa haim | saima sainda dyunsa eka uttama paryataka sthana hai jo suryasta evam suryodaya ke darshaniya sthala ke rupa mem jana jata hai | saima sainda dyunsa manava ke lie prakriti ke sarvashreshtha upaharom mem se eka hai | jaisalamera ke thara marusthala kshetra mem sthita saima sainda dyunsa sabase adhika prasiddha reta ke tilom mem se hai jo sala -bhara paryatakom ki bahuta bada़i sankhya ko akarshita karata hai | manavara marusthala bahari sahasika karanamom evam romancha ke lie sarvottama hai | paryataka apane apa ko adivasi gamvom ki yatra, marusthala ke vanya jivana ka darshana aura sthaniya shilpakarom ke shilpa ko dekhane mem vyasta rakha sakate haim | bikanera rajasthana ke uttara mem sthita hai aura samanyataya unta- pradesha ke rupa mem jana jata hai | vishva ki dasa pramukha shanadara relagada़iyom mem se eka, pailesa ऑna hvilsa, bharata ki shana kahi ja sakati hai jo yatra ki apani anupama, atulaniya, avismaraniya evam advitiya shaili ke lie jani jati hai | rajasthana ki rajasi bhumi ke darshana karane ke sarvottama madhyamom mem se eka, pailesa ऑna vhilsa bharata ke preranadayaka, atulaniya, avishvasaniya, prabhavashali evam preranatmaka kshetra se gujarati hai | bharatiya rela evam rajasthana paryataka vikasa nigama ke sanyukta prayasom se bharata ki pahali avakasha - relagada़i, pailesa ऑna vhilsa ne 1982 mem apani rajasi yatra arambha ki thi| aja yaha bhavya pailesa ऑna vhilsa relagada़i rajasi rajasthana ka anivarya hissa aura bharata ka gaurava ho gai hai jo rajasthana ke kuchha sarvottama paryataka - sthalom ke darshana karati hai | yaha rajasi yatra mai se agasta ke bicha ke mahinom ko chhoda़kara varsha - bhara hoti hai | vishva ki sarvottama romanchaka yatraom mem se eka pailesa ऑna vhilsa yatra ke daurana vishva stara ki sevaem evam suvidhaem pradana karati hai yaha rajasi yatra dilli se arambha hoti hai aura jayapura - jaisalamera - jodhapura - savai madhopura- chittauda़gadha़ - udayapura - bharatapura - agara hote hue dilli mem akara samapta hoti hai | pushkara mela pratyeka varsha hindu maha kartika (aktubara - navambara) ki shubha purnima ke avasara para lagata hai | rajasthana - bhavya, prakritika aura vastushilpiya ashcharyom se vibhushita pradesha | alabarta hala sangrahalaya jayapura, rajya ka sabase purana sangrahalaya mana jata hai |
paryatana vibhaga ne paryatana sankula ke nirmana ke lie kai nae sthanom ko nirdeshita kiya hai | hariyana paryatana ne hariyana ke sabhi kshetrom mem hotalom, motalom aura bhojanalayom ka prabandha kiya hai | panipata yamuna nadi ke tata para sthita hai aura yaham panipata ke tina aitihasika yuddha lada़e gae the | panipata eka audyogika shahara hai aura hathakaragha utpadom ke lie jana jata hai | jaham taka pata laga hai ‘danavira karna ka shahara’ karanala, divarom se ghira hua shahara raha hai aura shayada kisi samaya mem eka durga raha hoga| karanala jutom, krishi anusandhana sansthanom aura basamati chavala ke lie prasiddha hai|
panchakula jile mem uposhnakatibandhiya mahadvipiya manasuni jalavayu hai jaham hamem ritu avartana, garama grishmaritu, thandi shitaritu, aniyamita varsha aura tapamana mem atyadhika parivartana milata hai|
paryatana karane ke lie atyanta manorama sthalom mem se kuchha haim, kinnaura jile mem sangala ghati aura kalpa, shimala jile mem naladehara, narakanda aura sarahana, kullu jile mem manali aura manikarana, kangada़a jile mem dharmashala tatha lahaula aura spiti mem tabo | palamapura apane krishi mahavidyalaya aura chaya baganom ke lie prasiddha hai | himachala pradesha vanya-jivana premiyom, pakshi prekshakom tatha sahasika paryatakom ke saira karane ke lie bahuta achchhi jagaha hai | shimala paryatakom mem eka lokapriya shahara hai | shimala ki saira karane ke lie sala ke sarvottama mahine marcha - juna (vasanta), sitambara - aktubara (sharada) aura disambara - janavari (shitakalina himapata ka anubhava lene ke lie) haim | himachchhadita himalaya shrinkhala shimala se dikhai pada़ti hai | rajasthana apane paryatakom ke lie vividha prakara ke bhojana prastuta karata hai- yaha vividhata isaki sanskriti, isaki paitrika sanrachana, isaki bhaugolika sthiti aura isaki jalavayu jaisi vividha hai | da raॉyala kairija़ma ऑpha rajasthana tura (rajasi akarshana ki rajasthana yatra) ise sansara ke ati vanchhita paryataka gantavya karara deta hai | bharata ki prakritika sampada usaki sanskritika sampada ki taraha sanriddha aura vividha hai | yaham ke vanya-jiva abhayaranya aura rashtriya udyana apako vahi sara dete haim | raॉyala bangala taigara isa sanriddha kshetra mem basa hai, (jo) raॉyala taigara kaita ka ekamatra nivasa-sthana (hai)| bangala taigara sabase teja़ dauda़ne vala janavara hai | agara apa raॉyala bangala taigara mem dilachaspi rakhate haim tatha use usake svabhavika parivesha mem dekhana chahate haim, (to) ina bagha rakshita sthanom ki yatra avashya anandadayaka hogi | karbeta ka karbeta neshanala parka baghom aura satha hi unake shikara ke lie bhi ekaashraya-sthala hai, jinamem chara prakara ke hirana, jangali suara aura kuchha kama prasiddha janavara sammilita haim | svarna mandira sikkhom ke pavitra nagara anritasara mem sthita hai, jo kabhi ghana jangala tha aura gurunanaka ka nivasa-sthala tha | maidanom ke upara sthita dharmashala ghane chida़ ke peda़om aura devadara ke jangalom se ghiri hai| bahuta si jala dharaom ke satha pasa ki hima-rekha aura shitala svastha vatavarana parivesha ko bahuta hi lubhavana banata hai| kabhi shasana ka eka mahatvapurna kendra mana jane vala chanda vansha ka rajadhani shahara kangada़a, gaurava ki eka gatha kahata hai, jo itihasa mem kho gaya hai| ninna himalaya ki manorama ghatiyom mem se eka, utkrishta dhauladhara pahadiyom dvara parirakshita yaha ghati hari-bhari aura sanriddha hai | sikkima ke parvatiya kshetra mem hare-bhare pahada़i rastom, stupom, mathom aura mandirom ke satha-satha romancha ki prachurata hai | sanriddha sanskriti aura parampara ka parichaya prapta kijie jo ki apane asimita anandaprada sanskarom evam utsavom ke karana purnatah vilakshana evam mohaka hai | sikkima eka sukhadayi yatra pradana karata hai – romancha aura khoja ka eka paryatana | bahuta lambe samaya se manava apane hridaya mem eka chida़iya ki bhanti vishala nile akasha mem uda़ne ki gahari chaha lie hue hai, kuchha hada taka hama manava khule akasha mem uda़ne ke lie havai - khela ke romancha ko apanate hue apani isa pyasa ko buja pane mem sakshama haim | 26,000 phuta ki unchai para barpha se dhaki chotiyom ke shikhara vala himalaya vishva ki sarvashreshtha parvata shrrinkhala hai|
bharata mem paidala - yatra eka avismaraniya anubhava hai kyom ki yaha apako prakriti dvara nirmita ajnata rastom ka anubhava hi nahim deti,(balki) paidala - yatra paryataka ko desha aura isake logom ke satha sidhe samparka mem lati hai aura apako parvatiya vatavarana ko samajane mem sahayata karati hai | himalaya vishva ki sabase taruna parvata shrrinkhalaom mem se eka hai aura vishva ke sarvashreshtha parvatarohana kshetrom mem se gina jata hai| bharatiya himalaya shrrinkhala nissandeha vishva ki sabase bhavya aura prabhavashali parvata shrrinkhalaom mem se eka hai| thara registana mem sahasika yatra aisi yatra hai jise apa apane pure jivana mem nahim bhula paenge| paryataka usi registani anubhava ka ananda jaisalamera, rajasthana mem saima sainda dyunsa mem bhi le sakate haim jaham paryatakom ke manoranjana ke lie rajasthana paryatana vibhaga dvara sayankala mem sanskritika pradarshanom ka ayojana kiya jata hai. gira rashtriya udyana sagauna, gulamohara, khaira aura baragada vrikshom ka eka mila-jula patajada़i vana hai| satha hi gira rashtriya udyana bharata ke kisi bhi udyana ki tulana mem chite ki sabase adhika abadi vala avasa hai| bharata mem bahuta se bagha arakshita kshetra haim, jo isa khunkhara janavara ko surakshita rakha rahe haim, lekina jitani bahutayata aura niyamitata se inhem kanha rashtriya udyana mem dekha ja sakata hai vaisa aura kahim nahim | vishva ke sarvottama pakshi udyanom mem se eka, bharatapura pakshi abhayavana (kevaladeva ghana neshanala parka) eka arakshita kshetra hai jo pashu-varga ko bhi sanrakshana deta hai| udyana dekhane vale adhikansha paryatakom ka mukhya akarshana bahuta se pravasi pakshi hote haim jo saiberiya aura madhya eshiya jaise sudura pradeshom se ate haim aura apane prajanana sthanom ko lautane se pahale ve apani thanda bharatapura mem bitate haim | bharatapura abhayavana ke bada bharatapura shasakiya sangrahalaya mukhya dhyana khinchane valom mem se eka hai, jo bharatapura ke bhutakalina rajasi vaibhava ki jalaki pradana karata hai | sariska udyana tendue, jangali kutte, vana billi, lakada़bagghe, bheda़iye aura bagha ke satha-satha bahuta se mansahari pashuom ka ghara hai| sariska risasa bandarom ki adhika janasankhya ke lie suprasiddha hai, jo ki talavriksha ke asa-pasa adhika sankhya mem paye jate hai|
ranathambhaura rashtriya udyana projekta taigara rija़rvsa ऑpha da varlda (vishva ki bagha sanrakshana pariyojana) ke antargata sammilita hai aura apani bagha janasankhya ke lie adhika lokapriya hai| bagha ko usake prakritika avasa mem dekhana pratyeka vanyajivana samarthaka ki sabase vanchhaniya kalpana hai, jo ki ranathambhaura ke vana mem purna ho sakati hai| ranathambhaura vanyajivana udyana vishva ke kuchha una darshaniya sthanom mem se hai jaham baghom ka varchasva hai| ranathambhaura vanyajivana udyana daladali magaramachchha aura ubhayachara pashuom ke satha-satha chida़iyom ki 272 jatiyom aura sarisripom ka eka avasa hai| ranathambhaura vana ke chhupe hue khajane ki khoja ke lie jipa yatra bhi upalabdha karata hai, kyonki yaha raॉyala taigara ki khoja ke lie sarvottama sadhana hai| vanyajivana rashtriya udyana mem anya akarshana ranathambhaura kile ke khandahara aura jogi mahala hai jo ki 10vim shatabdi ko dinankita karate hai| aba vana vishrama griha mem parivartita jogi mahala bharata mem dusare sabase bada़e baragada ke vriksha ke lie prasiddha hai| taja nissandeha vishva ke sabase bhavya bhavanom mem se eka hai| apani sthapatya bhavyata aura kalatmaka sundarata ke lie prasiddha taja manava ki sabase gauravashali rachanaom mem gina jata hai aura vishva ke shreshtha ashcharyom ki suchi mem nirapavada rupa se sammilita hai| bharatiya pratishthita kavi taigora dvara " shashvatata ke chehare para ansu" ke rupa mem varnita, tajamahala nissandeha mugala vastukala ki parakashtha hai aura nitanta saralata se vishva ke ati ashcharyajanaka bhavanom mem se eka hai| agara apani sthapatya bhavyata ki sanriddhi, baja़arom aura abhushanom ke satha vishva ke ati utkrishta shaharom mem se hai| taja mahotsava ke samaya shilpagrama shahara mem pure vishva ke paryataka ekatra hote haim aura isaka bharapura ananda lete haim| agara ane vale paryataka taja ko apani yadom mem banae rakhane ke lie apane satha isaka eka namuna avashya lem jaenge| agara mem chamada़e ke thailom, kasidakari vale jutom ki kharidadari ka bhi bada़a akarshana hai aura petha (mithai) to nishchita rupa se agara ki eka visheshata hai| yadyapi sudura mem sthita hai, phira bhi khajuraho mandira sankula sthala videshi aura bharatiya donom paryatakom ke lie sarvadhika lokapriya sthanom mem se eka hai| khajuraho ke mandira apani murti kala ke satha paryataka ka dhyana banae rakhate haim, jo ki bahuta utkarsha aura jatila hai, jise aba koi bhi hubahu banane ka sapana bhi nahim dekha sakata| nishpadana mem shreshtha aura bhavabhivyakti mem utkrishta khajuraho mandira naritva ko samarpana haim| kalakara ki srijanatmaka pravrittiyom ne jivana ke vibhinna pahaluom aura bhavom ko patthara mem sundarata se utara hai|'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(25)

[['trevarsa', 'tainka', 'kabutara', 'mora', 'titara', 'jaisi', 'vibhinna', 'jatiyom', 'pakshiyom'], [], ['pakshi', 'avalokana', 'prasiddha'], ['guru', 'shikhara', 'maunta'], ['abu'], [], ['unchi', 'choti', 'pure', 'shahara'], ['achchha', 'naja़ara', 'dikhata', 'parvata'], ['shikhara', 'sthita'], ['chhota', 'sa', 'shaiva', 'punya', 'sthala'], ['dattatreya'], ['mandira', 'bhramana'], ['mahatvapurna', 'sthana', 'rajasthana'], ['thara', 'marusthala', 'asali', 'jadu', 'anubhava', 'pane', 'achchha', 'tarika', 'marusthala', 'sapha़ari', 'maunta'], ['abu', 'rajasthali'], ['rajasthana', 'shasakiya', 'hastashilpa', 'emporiyama', 'khadi', 'bhandara', 'kharidari'], ['mahatvapurna', 'sthana'], [], ['apraila'], ['juna', 'aktubara'], ['navambara', 'samaya', 'shahara'], ['bhramana'], ['achchha'], ['sahasika', 'karyom'], ['talasha', 'nikale', 'paryatakom', 'thara', 'marusthala', 'andaruni', 'hissom', 'ghumane', 'shaharom', 'aitihasika', 'khandaharom'], ['dekhane'], ['unta', 'sapha़ari', 'sarvottama', '


('trevarsa', 'tainka')
('trevarsa', 'kabutara')
('trevarsa', 'mora')
('tainka', 'kabutara')
('tainka', 'mora')
('tainka', 'titara')
('kabutara', 'mora')
('kabutara', 'titara')
('kabutara', 'jaisi')
('mora', 'titara')
('mora', 'jaisi')
('mora', 'vibhinna')
('titara', 'jaisi')
('titara', 'vibhinna')
('titara', 'jatiyom')
('jaisi', 'vibhinna')
('jaisi', 'jatiyom')
('jaisi', 'pakshiyom')
('vibhinna', 'jatiyom')
('vibhinna', 'pakshiyom')
('jatiyom', 'pakshiyom')
('pakshi', 'avalokana')
('pakshi', 'prasiddha')
('avalokana', 'prasiddha')
('guru', 'shikhara')
('guru', 'maunta')
('shikhara', 'maunta')
('unchi', 'choti')
('unchi', 'pure')
('unchi', 'shahara')
('choti', 'pure')
('choti', 'shahara')
('pure', 'shahara')
('achchha', 'naja़ara')
('achchha', 'dikhata')
('achchha', 'parvata')
('naja़ara', 'dikhata')
('naja़ara', 'parvata')
('dikhata', 'parvata')
('shikhara', 'sthita')
('chhota', 'sa')
('chhota', 'shaiva')
('chhota', 'punya')
('sa', 'shaiva')
('sa', 'punya')
('sa', 'sthala')
('shaiva',

('dilli', 'akara')
('dilli', 'samapta')
('akara', 'samapta')
('pushkara', 'mela')
('pushkara', 'pratyeka')
('pushkara', 'varsha')
('mela', 'pratyeka')
('mela', 'varsha')
('mela', 'hindu')
('pratyeka', 'varsha')
('pratyeka', 'hindu')
('pratyeka', 'maha')
('varsha', 'hindu')
('varsha', 'maha')
('varsha', 'kartika')
('hindu', 'maha')
('hindu', 'kartika')
('hindu', 'aktubara')
('maha', 'kartika')
('maha', 'aktubara')
('maha', 'navambara')
('kartika', 'aktubara')
('kartika', 'navambara')
('aktubara', 'navambara')
('shubha', 'purnima')
('avasara', 'lagata')
('rajasthana', 'bhavya')
('rajasthana', 'prakritika')
('rajasthana', 'vastushilpiya')
('bhavya', 'prakritika')
('bhavya', 'vastushilpiya')
('prakritika', 'vastushilpiya')
('ashcharyom', 'vibhushita')
('ashcharyom', 'pradesha')
('vibhushita', 'pradesha')
('alabarta', 'hala')
('alabarta', 'sangrahalaya')
('alabarta', 'jayapura')
('hala', 'sangrahalaya')
('hala', 'jayapura')
('sangrahalaya', 'jayapura')
('purana', 'sangrahalaya')
('purana', 

('sarvashreshtha', 'parvata')
('sarvashreshtha', 'shrrinkhala')
('sarvashreshtha', 'bharata')
('parvata', 'shrrinkhala')
('parvata', 'bharata')
('shrrinkhala', 'bharata')
('paidala', 'yatra')
('paidala', 'avismaraniya')
('yatra', 'avismaraniya')
('anubhava', 'kyom')
('anubhava', 'apako')
('anubhava', 'prakriti')
('kyom', 'apako')
('kyom', 'prakriti')
('kyom', 'nirmita')
('apako', 'prakriti')
('apako', 'nirmita')
('apako', 'ajnata')
('prakriti', 'nirmita')
('prakriti', 'ajnata')
('prakriti', 'rastom')
('nirmita', 'ajnata')
('nirmita', 'rastom')
('ajnata', 'rastom')
('deti,(balki', 'paidala')
('deti,(balki', 'yatra')
('deti,(balki', 'paryataka')
('paidala', 'yatra')
('paidala', 'paryataka')
('yatra', 'paryataka')
('desha', 'logom')
('sidhe', 'samparka')
('sidhe', 'lati')
('sidhe', 'apako')
('samparka', 'lati')
('samparka', 'apako')
('samparka', 'parvatiya')
('lati', 'apako')
('lati', 'parvatiya')
('lati', 'vatavarana')
('apako', 'parvatiya')
('apako', 'vatavarana')
('apako', 'samajane')


('vanchhaniya', 'ranathambhaura')
('kalpana', 'ranathambhaura')
('vana', 'purna')
('sakati', 'ranathambhaura')
('sakati', 'vanyajivana')
('sakati', 'udyana')
('ranathambhaura', 'vanyajivana')
('ranathambhaura', 'udyana')
('ranathambhaura', 'vishva')
('vanyajivana', 'udyana')
('vanyajivana', 'vishva')
('udyana', 'vishva')
('darshaniya', 'sthanom')
('baghom', 'varchasva')
('baghom', 'ranathambhaura')
('baghom', 'vanyajivana')
('varchasva', 'ranathambhaura')
('varchasva', 'vanyajivana')
('varchasva', 'udyana')
('ranathambhaura', 'vanyajivana')
('ranathambhaura', 'udyana')
('ranathambhaura', 'daladali')
('vanyajivana', 'udyana')
('vanyajivana', 'daladali')
('vanyajivana', 'magaramachchha')
('udyana', 'daladali')
('udyana', 'magaramachchha')
('udyana', 'ubhayachara')
('daladali', 'magaramachchha')
('daladali', 'ubhayachara')
('daladali', 'pashuom')
('magaramachchha', 'ubhayachara')
('magaramachchha', 'pashuom')
('magaramachchha', 'chida़iyom')
('ubhayachara', 'pashuom')
('ubhayachara', 'chi

Length: 1990, dtype: int64
rajasthana - 6.564731103736328
paryataka - 5.096152212715157
vishva - 4.915333246789211
udyana - 4.870369471755061
yatra - 4.061527719505592
shahara - 3.6508385604678986
sarvottama - 3.6050943237229087
sthita - 3.5473565696389393
bagha - 3.508939155267557
marusthala - 3.4695960687409717
jile - 3.3528236439154906
sapha़ari - 3.2748889052795827
agara - 3.171012892634582
romancha - 2.9537906645908603
sthala - 2.946658242888159
paryatakom - 2.937465890286088
ranathambhaura - 2.9197965141840485
bharata - 2.901455979463883
jata - 2.821059783779121
anubhava - 2.812166834120058
kshetra - 2.730958329630028
jivana - 2.596469511700528
sthanom - 2.5904880079117745
sanskriti - 2.4286060660931335
bhavya - 2.4183526255443373
rajasi - 2.3977802647167072
himalaya - 2.396696564242292


In [4]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =[' - ', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']

        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
        #print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
            #print(word)
        token_pairs_freq = pd.Series(token_pairs).value_counts()
        for i in token_pairs_freq:
            print(i)
        print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''trevarsa tainka kabutara, mora aura titara jaisi vibhinna jatiyom ke pakshiyom ke karana pakshi - avalokana ke lie prasiddha hai | guru shikhara maunta abu ki sabase unchi choti hai jaham se pure shahara ka bahuta achchha naja़ara dikhata hai| parvata ke shikhara para sthita eka chhota-sa shaiva punya-sthala aura dattatreya ka mandira bhramana ke do mahatvapurna sthana haim| rajasthana ke thara marusthala ke asali jadu ka anubhava pane ka sabase achchha tarika hai marusthala sapha़ari | maunta abu mem rajasthali, rajasthana shasakiya hastashilpa emporiyama aura khadi bhandara kharidari ke mahatvapurna sthana haim | apraila se juna aura aktubara se navambara ka samaya isa shahara ke bhramana ke lie sabase achchha hai | sahasika karyom ki talasha mem nikale paryatakom ke lie thara marusthala ke andaruni hissom mem ghumane, shaharom aura aitihasika khandaharom ko dekhane ke lie unta sapha़ari sarvottama vikalpa hai | rajasthana ke chhote- chhote gamvom mem ghumane ke anokhe anubhava ke lie unta sapha़ari eka romanchaka aura jokhima -bhara sadhana hai | thara marusthala ke khule bhubhaga para ghumane lie jipa sapha़ari eka adbhuta sadhana hai aura rajasthana ke khubasurata, shanta aura ranga-birange ilake ko dekhane ka anokha avasara pradana karati hai | ananda aura romancha ke mishrana ke satha rajasthana ke thara marusthala mem jipa sapha़ari vaham ke kuchha shanadara gantavyom ko samavishta karati hai| thara marusthala ke bhavya retile tile rajasthana ki anokhi dena haim jo ki vaham ki ranga-birangi sanskriti,vanya - jivana, aitihasika smarakom, kilom, udyanom aura jilom se alaga haim | insani hunara aura acharana se anachhue rajasthana ke ye retile tile prakriti ka eka uttama upahara haim jo registana ka eka jadui naja़ara pesha karate haim | nile asamana ke niche hava se akara mem dhale retile tile duniya -bhara se bada़i sankhya mem paryatakom ko akarshita karate haim aura rajasthana ki koi bhi saira ina retile tilom ki saira ke bina adhuri hai | retile tilom dvara bani laharem photographa़rom ke lie pha़oto khinchane ke lie uttama sthana rachati haim | rajasthana ke marusthala ki saira karane ka sarvottama tarika unta sapha़ari ke madhyama se hai, jo registani paryatana ka sabase lokapriya sadhana hai | rajasthana ke andaruni hisse sirpha़ isi sadhana ke jarie dekhe ja sakate haim jise prayah ‘registana ka jahaja’ kaha jata hai | rajasthana mem unta sapha़ari solahavim shatabdi se prachalana mem hai jise taba mukhya rupa se samana dhone ke lie istemala kiya jata tha, aura aba yaha manoranjana ka mahatvapurna sadhana hai | rajasthana ke thara marusthala mem chalane vale unta sapha़ari ke aja ke rupa mem prachina unta- karavam ka jadu dekha ja sakata hai | rajasthana - bhramana ka eka mahatvapurna hissa, unta sapha़ari paramparika aura ranga-birange rajasthana ki andaruni janki dikhata hai |
jaisalamera, jodhapura aura bikanera retile tilom ke lie jane jate haim para inamem se sabase lokapriya jaisalamera hai | adhikatara reta ke tile jaisalamera ke asapasa haim | saima sainda dyunsa eka uttama paryataka sthana hai jo suryasta evam suryodaya ke darshaniya sthala ke rupa mem jana jata hai | saima sainda dyunsa manava ke lie prakriti ke sarvashreshtha upaharom mem se eka hai | jaisalamera ke thara marusthala kshetra mem sthita saima sainda dyunsa sabase adhika prasiddha reta ke tilom mem se hai jo sala -bhara paryatakom ki bahuta bada़i sankhya ko akarshita karata hai | manavara marusthala bahari sahasika karanamom evam romancha ke lie sarvottama hai | paryataka apane apa ko adivasi gamvom ki yatra, marusthala ke vanya jivana ka darshana aura sthaniya shilpakarom ke shilpa ko dekhane mem vyasta rakha sakate haim | bikanera rajasthana ke uttara mem sthita hai aura samanyataya unta- pradesha ke rupa mem jana jata hai | vishva ki dasa pramukha shanadara relagada़iyom mem se eka, pailesa ऑna hvilsa, bharata ki shana kahi ja sakati hai jo yatra ki apani anupama, atulaniya, avismaraniya evam advitiya shaili ke lie jani jati hai | rajasthana ki rajasi bhumi ke darshana karane ke sarvottama madhyamom mem se eka, pailesa ऑna vhilsa bharata ke preranadayaka, atulaniya, avishvasaniya, prabhavashali evam preranatmaka kshetra se gujarati hai | bharatiya rela evam rajasthana paryataka vikasa nigama ke sanyukta prayasom se bharata ki pahali avakasha - relagada़i, pailesa ऑna vhilsa ne 1982 mem apani rajasi yatra arambha ki thi| aja yaha bhavya pailesa ऑna vhilsa relagada़i rajasi rajasthana ka anivarya hissa aura bharata ka gaurava ho gai hai jo rajasthana ke kuchha sarvottama paryataka - sthalom ke darshana karati hai | yaha rajasi yatra mai se agasta ke bicha ke mahinom ko chhoda़kara varsha - bhara hoti hai | vishva ki sarvottama romanchaka yatraom mem se eka pailesa ऑna vhilsa yatra ke daurana vishva stara ki sevaem evam suvidhaem pradana karati hai yaha rajasi yatra dilli se arambha hoti hai aura jayapura - jaisalamera - jodhapura - savai madhopura- chittauda़gadha़ - udayapura - bharatapura - agara hote hue dilli mem akara samapta hoti hai | pushkara mela pratyeka varsha hindu maha kartika (aktubara - navambara) ki shubha purnima ke avasara para lagata hai | rajasthana - bhavya, prakritika aura vastushilpiya ashcharyom se vibhushita pradesha | alabarta hala sangrahalaya jayapura, rajya ka sabase purana sangrahalaya mana jata hai |
paryatana vibhaga ne paryatana sankula ke nirmana ke lie kai nae sthanom ko nirdeshita kiya hai | hariyana paryatana ne hariyana ke sabhi kshetrom mem hotalom, motalom aura bhojanalayom ka prabandha kiya hai | panipata yamuna nadi ke tata para sthita hai aura yaham panipata ke tina aitihasika yuddha lada़e gae the | panipata eka audyogika shahara hai aura hathakaragha utpadom ke lie jana jata hai | jaham taka pata laga hai ‘danavira karna ka shahara’ karanala, divarom se ghira hua shahara raha hai aura shayada kisi samaya mem eka durga raha hoga| karanala jutom, krishi anusandhana sansthanom aura basamati chavala ke lie prasiddha hai|
panchakula jile mem uposhnakatibandhiya mahadvipiya manasuni jalavayu hai jaham hamem ritu avartana, garama grishmaritu, thandi shitaritu, aniyamita varsha aura tapamana mem atyadhika parivartana milata hai|
paryatana karane ke lie atyanta manorama sthalom mem se kuchha haim, kinnaura jile mem sangala ghati aura kalpa, shimala jile mem naladehara, narakanda aura sarahana, kullu jile mem manali aura manikarana, kangada़a jile mem dharmashala tatha lahaula aura spiti mem tabo | palamapura apane krishi mahavidyalaya aura chaya baganom ke lie prasiddha hai | himachala pradesha vanya-jivana premiyom, pakshi prekshakom tatha sahasika paryatakom ke saira karane ke lie bahuta achchhi jagaha hai | shimala paryatakom mem eka lokapriya shahara hai | shimala ki saira karane ke lie sala ke sarvottama mahine marcha - juna (vasanta), sitambara - aktubara (sharada) aura disambara - janavari (shitakalina himapata ka anubhava lene ke lie) haim | himachchhadita himalaya shrinkhala shimala se dikhai pada़ti hai | rajasthana apane paryatakom ke lie vividha prakara ke bhojana prastuta karata hai- yaha vividhata isaki sanskriti, isaki paitrika sanrachana, isaki bhaugolika sthiti aura isaki jalavayu jaisi vividha hai | da raॉyala kairija़ma ऑpha rajasthana tura (rajasi akarshana ki rajasthana yatra) ise sansara ke ati vanchhita paryataka gantavya karara deta hai | bharata ki prakritika sampada usaki sanskritika sampada ki taraha sanriddha aura vividha hai | yaham ke vanya-jiva abhayaranya aura rashtriya udyana apako vahi sara dete haim | raॉyala bangala taigara isa sanriddha kshetra mem basa hai, (jo) raॉyala taigara kaita ka ekamatra nivasa-sthana (hai)| bangala taigara sabase teja़ dauda़ne vala janavara hai | agara apa raॉyala bangala taigara mem dilachaspi rakhate haim tatha use usake svabhavika parivesha mem dekhana chahate haim, (to) ina bagha rakshita sthanom ki yatra avashya anandadayaka hogi | karbeta ka karbeta neshanala parka baghom aura satha hi unake shikara ke lie bhi ekaashraya-sthala hai, jinamem chara prakara ke hirana, jangali suara aura kuchha kama prasiddha janavara sammilita haim | svarna mandira sikkhom ke pavitra nagara anritasara mem sthita hai, jo kabhi ghana jangala tha aura gurunanaka ka nivasa-sthala tha | maidanom ke upara sthita dharmashala ghane chida़ ke peda़om aura devadara ke jangalom se ghiri hai| bahuta si jala dharaom ke satha pasa ki hima-rekha aura shitala svastha vatavarana parivesha ko bahuta hi lubhavana banata hai| kabhi shasana ka eka mahatvapurna kendra mana jane vala chanda vansha ka rajadhani shahara kangada़a, gaurava ki eka gatha kahata hai, jo itihasa mem kho gaya hai| ninna himalaya ki manorama ghatiyom mem se eka, utkrishta dhauladhara pahadiyom dvara parirakshita yaha ghati hari-bhari aura sanriddha hai | sikkima ke parvatiya kshetra mem hare-bhare pahada़i rastom, stupom, mathom aura mandirom ke satha-satha romancha ki prachurata hai | sanriddha sanskriti aura parampara ka parichaya prapta kijie jo ki apane asimita anandaprada sanskarom evam utsavom ke karana purnatah vilakshana evam mohaka hai | sikkima eka sukhadayi yatra pradana karata hai – romancha aura khoja ka eka paryatana | bahuta lambe samaya se manava apane hridaya mem eka chida़iya ki bhanti vishala nile akasha mem uda़ne ki gahari chaha lie hue hai, kuchha hada taka hama manava khule akasha mem uda़ne ke lie havai - khela ke romancha ko apanate hue apani isa pyasa ko buja pane mem sakshama haim | 26,000 phuta ki unchai para barpha se dhaki chotiyom ke shikhara vala himalaya vishva ki sarvashreshtha parvata shrrinkhala hai|
bharata mem paidala - yatra eka avismaraniya anubhava hai kyom ki yaha apako prakriti dvara nirmita ajnata rastom ka anubhava hi nahim deti,(balki) paidala - yatra paryataka ko desha aura isake logom ke satha sidhe samparka mem lati hai aura apako parvatiya vatavarana ko samajane mem sahayata karati hai | himalaya vishva ki sabase taruna parvata shrrinkhalaom mem se eka hai aura vishva ke sarvashreshtha parvatarohana kshetrom mem se gina jata hai| bharatiya himalaya shrrinkhala nissandeha vishva ki sabase bhavya aura prabhavashali parvata shrrinkhalaom mem se eka hai| thara registana mem sahasika yatra aisi yatra hai jise apa apane pure jivana mem nahim bhula paenge| paryataka usi registani anubhava ka ananda jaisalamera, rajasthana mem saima sainda dyunsa mem bhi le sakate haim jaham paryatakom ke manoranjana ke lie rajasthana paryatana vibhaga dvara sayankala mem sanskritika pradarshanom ka ayojana kiya jata hai. gira rashtriya udyana sagauna, gulamohara, khaira aura baragada vrikshom ka eka mila-jula patajada़i vana hai| satha hi gira rashtriya udyana bharata ke kisi bhi udyana ki tulana mem chite ki sabase adhika abadi vala avasa hai| bharata mem bahuta se bagha arakshita kshetra haim, jo isa khunkhara janavara ko surakshita rakha rahe haim, lekina jitani bahutayata aura niyamitata se inhem kanha rashtriya udyana mem dekha ja sakata hai vaisa aura kahim nahim | vishva ke sarvottama pakshi udyanom mem se eka, bharatapura pakshi abhayavana (kevaladeva ghana neshanala parka) eka arakshita kshetra hai jo pashu-varga ko bhi sanrakshana deta hai| udyana dekhane vale adhikansha paryatakom ka mukhya akarshana bahuta se pravasi pakshi hote haim jo saiberiya aura madhya eshiya jaise sudura pradeshom se ate haim aura apane prajanana sthanom ko lautane se pahale ve apani thanda bharatapura mem bitate haim | bharatapura abhayavana ke bada bharatapura shasakiya sangrahalaya mukhya dhyana khinchane valom mem se eka hai, jo bharatapura ke bhutakalina rajasi vaibhava ki jalaki pradana karata hai | sariska udyana tendue, jangali kutte, vana billi, lakada़bagghe, bheda़iye aura bagha ke satha-satha bahuta se mansahari pashuom ka ghara hai| sariska risasa bandarom ki adhika janasankhya ke lie suprasiddha hai, jo ki talavriksha ke asa-pasa adhika sankhya mem paye jate hai|
ranathambhaura rashtriya udyana projekta taigara rija़rvsa ऑpha da varlda (vishva ki bagha sanrakshana pariyojana) ke antargata sammilita hai aura apani bagha janasankhya ke lie adhika lokapriya hai| bagha ko usake prakritika avasa mem dekhana pratyeka vanyajivana samarthaka ki sabase vanchhaniya kalpana hai, jo ki ranathambhaura ke vana mem purna ho sakati hai| ranathambhaura vanyajivana udyana vishva ke kuchha una darshaniya sthanom mem se hai jaham baghom ka varchasva hai| ranathambhaura vanyajivana udyana daladali magaramachchha aura ubhayachara pashuom ke satha-satha chida़iyom ki 272 jatiyom aura sarisripom ka eka avasa hai| ranathambhaura vana ke chhupe hue khajane ki khoja ke lie jipa yatra bhi upalabdha karata hai, kyonki yaha raॉyala taigara ki khoja ke lie sarvottama sadhana hai| vanyajivana rashtriya udyana mem anya akarshana ranathambhaura kile ke khandahara aura jogi mahala hai jo ki 10vim shatabdi ko dinankita karate hai| aba vana vishrama griha mem parivartita jogi mahala bharata mem dusare sabase bada़e baragada ke vriksha ke lie prasiddha hai| taja nissandeha vishva ke sabase bhavya bhavanom mem se eka hai| apani sthapatya bhavyata aura kalatmaka sundarata ke lie prasiddha taja manava ki sabase gauravashali rachanaom mem gina jata hai aura vishva ke shreshtha ashcharyom ki suchi mem nirapavada rupa se sammilita hai| bharatiya pratishthita kavi taigora dvara " shashvatata ke chehare para ansu" ke rupa mem varnita, tajamahala nissandeha mugala vastukala ki parakashtha hai aura nitanta saralata se vishva ke ati ashcharyajanaka bhavanom mem se eka hai| agara apani sthapatya bhavyata ki sanriddhi, baja़arom aura abhushanom ke satha vishva ke ati utkrishta shaharom mem se hai| taja mahotsava ke samaya shilpagrama shahara mem pure vishva ke paryataka ekatra hote haim aura isaka bharapura ananda lete haim| agara ane vale paryataka taja ko apani yadom mem banae rakhane ke lie apane satha isaka eka namuna avashya lem jaenge| agara mem chamada़e ke thailom, kasidakari vale jutom ki kharidadari ka bhi bada़a akarshana hai aura petha (mithai) to nishchita rupa se agara ki eka visheshata hai| yadyapi sudura mem sthita hai, phira bhi khajuraho mandira sankula sthala videshi aura bharatiya donom paryatakom ke lie sarvadhika lokapriya sthanom mem se eka hai| khajuraho ke mandira apani murti kala ke satha paryataka ka dhyana banae rakhate haim, jo ki bahuta utkarsha aura jatila hai, jise aba koi bhi hubahu banane ka sapana bhi nahim dekha sakata| nishpadana mem shreshtha aura bhavabhivyakti mem utkrishta khajuraho mandira naritva ko samarpana haim| kalakara ki srijanatmaka pravrittiyom ne jivana ke vibhinna pahaluom aura bhavom ko patthara mem sundarata se utara hai|'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(25)

7
6
5
5
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [46]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =[' - ', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']

        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])})
        print(arr)                 
         
        #token_pairs_freq = pd.Series(token_pairs).value_counts()
        #for i in token_pairs_freq:
         #   print(i)
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''trevarsa tainka kabutara, mora aura titara jaisi vibhinna jatiyom ke pakshiyom ke karana pakshi - avalokana ke lie prasiddha hai | guru shikhara maunta abu ki sabase unchi choti hai jaham se pure shahara ka bahuta achchha naja़ara dikhata hai| parvata ke shikhara para sthita eka chhota-sa shaiva punya-sthala aura dattatreya ka mandira bhramana ke do mahatvapurna sthana haim| rajasthana ke thara marusthala ke asali jadu ka anubhava pane ka sabase achchha tarika hai marusthala sapha़ari | maunta abu mem rajasthali, rajasthana shasakiya hastashilpa emporiyama aura khadi bhandara kharidari ke mahatvapurna sthana haim | apraila se juna aura aktubara se navambara ka samaya isa shahara ke bhramana ke lie sabase achchha hai | sahasika karyom ki talasha mem nikale paryatakom ke lie thara marusthala ke andaruni hissom mem ghumane, shaharom aura aitihasika khandaharom ko dekhane ke lie unta sapha़ari sarvottama vikalpa hai | rajasthana ke chhote- chhote gamvom mem ghumane ke anokhe anubhava ke lie unta sapha़ari eka romanchaka aura jokhima -bhara sadhana hai | thara marusthala ke khule bhubhaga para ghumane lie jipa sapha़ari eka adbhuta sadhana hai aura rajasthana ke khubasurata, shanta aura ranga-birange ilake ko dekhane ka anokha avasara pradana karati hai | ananda aura romancha ke mishrana ke satha rajasthana ke thara marusthala mem jipa sapha़ari vaham ke kuchha shanadara gantavyom ko samavishta karati hai| thara marusthala ke bhavya retile tile rajasthana ki anokhi dena haim jo ki vaham ki ranga-birangi sanskriti,vanya - jivana, aitihasika smarakom, kilom, udyanom aura jilom se alaga haim | insani hunara aura acharana se anachhue rajasthana ke ye retile tile prakriti ka eka uttama upahara haim jo registana ka eka jadui naja़ara pesha karate haim | nile asamana ke niche hava se akara mem dhale retile tile duniya -bhara se bada़i sankhya mem paryatakom ko akarshita karate haim aura rajasthana ki koi bhi saira ina retile tilom ki saira ke bina adhuri hai | retile tilom dvara bani laharem photographa़rom ke lie pha़oto khinchane ke lie uttama sthana rachati haim | rajasthana ke marusthala ki saira karane ka sarvottama tarika unta sapha़ari ke madhyama se hai, jo registani paryatana ka sabase lokapriya sadhana hai | rajasthana ke andaruni hisse sirpha़ isi sadhana ke jarie dekhe ja sakate haim jise prayah ‘registana ka jahaja’ kaha jata hai | rajasthana mem unta sapha़ari solahavim shatabdi se prachalana mem hai jise taba mukhya rupa se samana dhone ke lie istemala kiya jata tha, aura aba yaha manoranjana ka mahatvapurna sadhana hai | rajasthana ke thara marusthala mem chalane vale unta sapha़ari ke aja ke rupa mem prachina unta- karavam ka jadu dekha ja sakata hai | rajasthana - bhramana ka eka mahatvapurna hissa, unta sapha़ari paramparika aura ranga-birange rajasthana ki andaruni janki dikhata hai |
jaisalamera, jodhapura aura bikanera retile tilom ke lie jane jate haim para inamem se sabase lokapriya jaisalamera hai | adhikatara reta ke tile jaisalamera ke asapasa haim | saima sainda dyunsa eka uttama paryataka sthana hai jo suryasta evam suryodaya ke darshaniya sthala ke rupa mem jana jata hai | saima sainda dyunsa manava ke lie prakriti ke sarvashreshtha upaharom mem se eka hai | jaisalamera ke thara marusthala kshetra mem sthita saima sainda dyunsa sabase adhika prasiddha reta ke tilom mem se hai jo sala -bhara paryatakom ki bahuta bada़i sankhya ko akarshita karata hai | manavara marusthala bahari sahasika karanamom evam romancha ke lie sarvottama hai | paryataka apane apa ko adivasi gamvom ki yatra, marusthala ke vanya jivana ka darshana aura sthaniya shilpakarom ke shilpa ko dekhane mem vyasta rakha sakate haim | bikanera rajasthana ke uttara mem sthita hai aura samanyataya unta- pradesha ke rupa mem jana jata hai | vishva ki dasa pramukha shanadara relagada़iyom mem se eka, pailesa ऑna hvilsa, bharata ki shana kahi ja sakati hai jo yatra ki apani anupama, atulaniya, avismaraniya evam advitiya shaili ke lie jani jati hai | rajasthana ki rajasi bhumi ke darshana karane ke sarvottama madhyamom mem se eka, pailesa ऑna vhilsa bharata ke preranadayaka, atulaniya, avishvasaniya, prabhavashali evam preranatmaka kshetra se gujarati hai | bharatiya rela evam rajasthana paryataka vikasa nigama ke sanyukta prayasom se bharata ki pahali avakasha - relagada़i, pailesa ऑna vhilsa ne 1982 mem apani rajasi yatra arambha ki thi| aja yaha bhavya pailesa ऑna vhilsa relagada़i rajasi rajasthana ka anivarya hissa aura bharata ka gaurava ho gai hai jo rajasthana ke kuchha sarvottama paryataka - sthalom ke darshana karati hai | yaha rajasi yatra mai se agasta ke bicha ke mahinom ko chhoda़kara varsha - bhara hoti hai | vishva ki sarvottama romanchaka yatraom mem se eka pailesa ऑna vhilsa yatra ke daurana vishva stara ki sevaem evam suvidhaem pradana karati hai yaha rajasi yatra dilli se arambha hoti hai aura jayapura - jaisalamera - jodhapura - savai madhopura- chittauda़gadha़ - udayapura - bharatapura - agara hote hue dilli mem akara samapta hoti hai | pushkara mela pratyeka varsha hindu maha kartika (aktubara - navambara) ki shubha purnima ke avasara para lagata hai | rajasthana - bhavya, prakritika aura vastushilpiya ashcharyom se vibhushita pradesha | alabarta hala sangrahalaya jayapura, rajya ka sabase purana sangrahalaya mana jata hai |
paryatana vibhaga ne paryatana sankula ke nirmana ke lie kai nae sthanom ko nirdeshita kiya hai | hariyana paryatana ne hariyana ke sabhi kshetrom mem hotalom, motalom aura bhojanalayom ka prabandha kiya hai | panipata yamuna nadi ke tata para sthita hai aura yaham panipata ke tina aitihasika yuddha lada़e gae the | panipata eka audyogika shahara hai aura hathakaragha utpadom ke lie jana jata hai | jaham taka pata laga hai ‘danavira karna ka shahara’ karanala, divarom se ghira hua shahara raha hai aura shayada kisi samaya mem eka durga raha hoga| karanala jutom, krishi anusandhana sansthanom aura basamati chavala ke lie prasiddha hai|
panchakula jile mem uposhnakatibandhiya mahadvipiya manasuni jalavayu hai jaham hamem ritu avartana, garama grishmaritu, thandi shitaritu, aniyamita varsha aura tapamana mem atyadhika parivartana milata hai|
paryatana karane ke lie atyanta manorama sthalom mem se kuchha haim, kinnaura jile mem sangala ghati aura kalpa, shimala jile mem naladehara, narakanda aura sarahana, kullu jile mem manali aura manikarana, kangada़a jile mem dharmashala tatha lahaula aura spiti mem tabo | palamapura apane krishi mahavidyalaya aura chaya baganom ke lie prasiddha hai | himachala pradesha vanya-jivana premiyom, pakshi prekshakom tatha sahasika paryatakom ke saira karane ke lie bahuta achchhi jagaha hai | shimala paryatakom mem eka lokapriya shahara hai | shimala ki saira karane ke lie sala ke sarvottama mahine marcha - juna (vasanta), sitambara - aktubara (sharada) aura disambara - janavari (shitakalina himapata ka anubhava lene ke lie) haim | himachchhadita himalaya shrinkhala shimala se dikhai pada़ti hai | rajasthana apane paryatakom ke lie vividha prakara ke bhojana prastuta karata hai- yaha vividhata isaki sanskriti, isaki paitrika sanrachana, isaki bhaugolika sthiti aura isaki jalavayu jaisi vividha hai | da raॉyala kairija़ma ऑpha rajasthana tura (rajasi akarshana ki rajasthana yatra) ise sansara ke ati vanchhita paryataka gantavya karara deta hai | bharata ki prakritika sampada usaki sanskritika sampada ki taraha sanriddha aura vividha hai | yaham ke vanya-jiva abhayaranya aura rashtriya udyana apako vahi sara dete haim | raॉyala bangala taigara isa sanriddha kshetra mem basa hai, (jo) raॉyala taigara kaita ka ekamatra nivasa-sthana (hai)| bangala taigara sabase teja़ dauda़ne vala janavara hai | agara apa raॉyala bangala taigara mem dilachaspi rakhate haim tatha use usake svabhavika parivesha mem dekhana chahate haim, (to) ina bagha rakshita sthanom ki yatra avashya anandadayaka hogi | karbeta ka karbeta neshanala parka baghom aura satha hi unake shikara ke lie bhi ekaashraya-sthala hai, jinamem chara prakara ke hirana, jangali suara aura kuchha kama prasiddha janavara sammilita haim | svarna mandira sikkhom ke pavitra nagara anritasara mem sthita hai, jo kabhi ghana jangala tha aura gurunanaka ka nivasa-sthala tha | maidanom ke upara sthita dharmashala ghane chida़ ke peda़om aura devadara ke jangalom se ghiri hai| bahuta si jala dharaom ke satha pasa ki hima-rekha aura shitala svastha vatavarana parivesha ko bahuta hi lubhavana banata hai| kabhi shasana ka eka mahatvapurna kendra mana jane vala chanda vansha ka rajadhani shahara kangada़a, gaurava ki eka gatha kahata hai, jo itihasa mem kho gaya hai| ninna himalaya ki manorama ghatiyom mem se eka, utkrishta dhauladhara pahadiyom dvara parirakshita yaha ghati hari-bhari aura sanriddha hai | sikkima ke parvatiya kshetra mem hare-bhare pahada़i rastom, stupom, mathom aura mandirom ke satha-satha romancha ki prachurata hai | sanriddha sanskriti aura parampara ka parichaya prapta kijie jo ki apane asimita anandaprada sanskarom evam utsavom ke karana purnatah vilakshana evam mohaka hai | sikkima eka sukhadayi yatra pradana karata hai – romancha aura khoja ka eka paryatana | bahuta lambe samaya se manava apane hridaya mem eka chida़iya ki bhanti vishala nile akasha mem uda़ne ki gahari chaha lie hue hai, kuchha hada taka hama manava khule akasha mem uda़ne ke lie havai - khela ke romancha ko apanate hue apani isa pyasa ko buja pane mem sakshama haim | 26,000 phuta ki unchai para barpha se dhaki chotiyom ke shikhara vala himalaya vishva ki sarvashreshtha parvata shrrinkhala hai|
bharata mem paidala - yatra eka avismaraniya anubhava hai kyom ki yaha apako prakriti dvara nirmita ajnata rastom ka anubhava hi nahim deti,(balki) paidala - yatra paryataka ko desha aura isake logom ke satha sidhe samparka mem lati hai aura apako parvatiya vatavarana ko samajane mem sahayata karati hai | himalaya vishva ki sabase taruna parvata shrrinkhalaom mem se eka hai aura vishva ke sarvashreshtha parvatarohana kshetrom mem se gina jata hai| bharatiya himalaya shrrinkhala nissandeha vishva ki sabase bhavya aura prabhavashali parvata shrrinkhalaom mem se eka hai| thara registana mem sahasika yatra aisi yatra hai jise apa apane pure jivana mem nahim bhula paenge| paryataka usi registani anubhava ka ananda jaisalamera, rajasthana mem saima sainda dyunsa mem bhi le sakate haim jaham paryatakom ke manoranjana ke lie rajasthana paryatana vibhaga dvara sayankala mem sanskritika pradarshanom ka ayojana kiya jata hai. gira rashtriya udyana sagauna, gulamohara, khaira aura baragada vrikshom ka eka mila-jula patajada़i vana hai| satha hi gira rashtriya udyana bharata ke kisi bhi udyana ki tulana mem chite ki sabase adhika abadi vala avasa hai| bharata mem bahuta se bagha arakshita kshetra haim, jo isa khunkhara janavara ko surakshita rakha rahe haim, lekina jitani bahutayata aura niyamitata se inhem kanha rashtriya udyana mem dekha ja sakata hai vaisa aura kahim nahim | vishva ke sarvottama pakshi udyanom mem se eka, bharatapura pakshi abhayavana (kevaladeva ghana neshanala parka) eka arakshita kshetra hai jo pashu-varga ko bhi sanrakshana deta hai| udyana dekhane vale adhikansha paryatakom ka mukhya akarshana bahuta se pravasi pakshi hote haim jo saiberiya aura madhya eshiya jaise sudura pradeshom se ate haim aura apane prajanana sthanom ko lautane se pahale ve apani thanda bharatapura mem bitate haim | bharatapura abhayavana ke bada bharatapura shasakiya sangrahalaya mukhya dhyana khinchane valom mem se eka hai, jo bharatapura ke bhutakalina rajasi vaibhava ki jalaki pradana karata hai | sariska udyana tendue, jangali kutte, vana billi, lakada़bagghe, bheda़iye aura bagha ke satha-satha bahuta se mansahari pashuom ka ghara hai| sariska risasa bandarom ki adhika janasankhya ke lie suprasiddha hai, jo ki talavriksha ke asa-pasa adhika sankhya mem paye jate hai|
ranathambhaura rashtriya udyana projekta taigara rija़rvsa ऑpha da varlda (vishva ki bagha sanrakshana pariyojana) ke antargata sammilita hai aura apani bagha janasankhya ke lie adhika lokapriya hai| bagha ko usake prakritika avasa mem dekhana pratyeka vanyajivana samarthaka ki sabase vanchhaniya kalpana hai, jo ki ranathambhaura ke vana mem purna ho sakati hai| ranathambhaura vanyajivana udyana vishva ke kuchha una darshaniya sthanom mem se hai jaham baghom ka varchasva hai| ranathambhaura vanyajivana udyana daladali magaramachchha aura ubhayachara pashuom ke satha-satha chida़iyom ki 272 jatiyom aura sarisripom ka eka avasa hai| ranathambhaura vana ke chhupe hue khajane ki khoja ke lie jipa yatra bhi upalabdha karata hai, kyonki yaha raॉyala taigara ki khoja ke lie sarvottama sadhana hai| vanyajivana rashtriya udyana mem anya akarshana ranathambhaura kile ke khandahara aura jogi mahala hai jo ki 10vim shatabdi ko dinankita karate hai| aba vana vishrama griha mem parivartita jogi mahala bharata mem dusare sabase bada़e baragada ke vriksha ke lie prasiddha hai| taja nissandeha vishva ke sabase bhavya bhavanom mem se eka hai| apani sthapatya bhavyata aura kalatmaka sundarata ke lie prasiddha taja manava ki sabase gauravashali rachanaom mem gina jata hai aura vishva ke shreshtha ashcharyom ki suchi mem nirapavada rupa se sammilita hai| bharatiya pratishthita kavi taigora dvara " shashvatata ke chehare para ansu" ke rupa mem varnita, tajamahala nissandeha mugala vastukala ki parakashtha hai aura nitanta saralata se vishva ke ati ashcharyajanaka bhavanom mem se eka hai| agara apani sthapatya bhavyata ki sanriddhi, baja़arom aura abhushanom ke satha vishva ke ati utkrishta shaharom mem se hai| taja mahotsava ke samaya shilpagrama shahara mem pure vishva ke paryataka ekatra hote haim aura isaka bharapura ananda lete haim| agara ane vale paryataka taja ko apani yadom mem banae rakhane ke lie apane satha isaka eka namuna avashya lem jaenge| agara mem chamada़e ke thailom, kasidakari vale jutom ki kharidadari ka bhi bada़a akarshana hai aura petha (mithai) to nishchita rupa se agara ki eka visheshata hai| yadyapi sudura mem sthita hai, phira bhi khajuraho mandira sankula sthala videshi aura bharatiya donom paryatakom ke lie sarvadhika lokapriya sthanom mem se eka hai| khajuraho ke mandira apani murti kala ke satha paryataka ka dhyana banae rakhate haim, jo ki bahuta utkarsha aura jatila hai, jise aba koi bhi hubahu banane ka sapana bhi nahim dekha sakata| nishpadana mem shreshtha aura bhavabhivyakti mem utkrishta khajuraho mandira naritva ko samarpana haim| kalakara ki srijanatmaka pravrittiyom ne jivana ke vibhinna pahaluom aura bhavom ko patthara mem sundarata se utara hai|'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(25)

SyntaxError: invalid syntax (<ipython-input-46-b9c21f46a168>, line 85)

In [10]:
a = ['hi', 'me', 'hate', 'love']
b = ['hi', 'me', 'hate', 'love', 'hi', 'me', 'hate', 'love', 'hi', 'me', 'hate', 'love', 'hi']
for i in a:
    count = 0
    for j in b:
        if i == j:
            count= count+1
        else:
            continue
    print(i, count)


hi 4
me 3
hate 3
love 3


In [30]:
arr = pd.Series(data = [0], index = [0])
a = pd.Series(data = [1,1,2,3], index = [0,1,2,3])
arr.append(a)


0    0
0    1
1    1
2    2
3    3
dtype: int64

In [31]:
a


0    1
1    1
2    2
3    3
dtype: int64

In [55]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =[' - ', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']

        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        for i in arr['Freq']:
            print(i)
         
        #token_pairs_freq = pd.Series(token_pairs).value_counts()
        #for i in token_pairs_freq:
         #   print(i)
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''trevarsa tainka kabutara, mora aura titara jaisi vibhinna jatiyom ke pakshiyom ke karana pakshi - avalokana ke lie prasiddha hai | guru shikhara maunta abu ki sabase unchi choti hai jaham se pure shahara ka bahuta achchha naja़ara dikhata hai| parvata ke shikhara para sthita eka chhota-sa shaiva punya-sthala aura dattatreya ka mandira bhramana ke do mahatvapurna sthana haim| rajasthana ke thara marusthala ke asali jadu ka anubhava pane ka sabase achchha tarika hai marusthala sapha़ari | maunta abu mem rajasthali, rajasthana shasakiya hastashilpa emporiyama aura khadi bhandara kharidari ke mahatvapurna sthana haim | apraila se juna aura aktubara se navambara ka samaya isa shahara ke bhramana ke lie sabase achchha hai | sahasika karyom ki talasha mem nikale paryatakom ke lie thara marusthala ke andaruni hissom mem ghumane, shaharom aura aitihasika khandaharom ko dekhane ke lie unta sapha़ari sarvottama vikalpa hai | rajasthana ke chhote- chhote gamvom mem ghumane ke anokhe anubhava ke lie unta sapha़ari eka romanchaka aura jokhima -bhara sadhana hai | thara marusthala ke khule bhubhaga para ghumane lie jipa sapha़ari eka adbhuta sadhana hai aura rajasthana ke khubasurata, shanta aura ranga-birange ilake ko dekhane ka anokha avasara pradana karati hai | ananda aura romancha ke mishrana ke satha rajasthana ke thara marusthala mem jipa sapha़ari vaham ke kuchha shanadara gantavyom ko samavishta karati hai| thara marusthala ke bhavya retile tile rajasthana ki anokhi dena haim jo ki vaham ki ranga-birangi sanskriti,vanya - jivana, aitihasika smarakom, kilom, udyanom aura jilom se alaga haim | insani hunara aura acharana se anachhue rajasthana ke ye retile tile prakriti ka eka uttama upahara haim jo registana ka eka jadui naja़ara pesha karate haim | nile asamana ke niche hava se akara mem dhale retile tile duniya -bhara se bada़i sankhya mem paryatakom ko akarshita karate haim aura rajasthana ki koi bhi saira ina retile tilom ki saira ke bina adhuri hai | retile tilom dvara bani laharem photographa़rom ke lie pha़oto khinchane ke lie uttama sthana rachati haim | rajasthana ke marusthala ki saira karane ka sarvottama tarika unta sapha़ari ke madhyama se hai, jo registani paryatana ka sabase lokapriya sadhana hai | rajasthana ke andaruni hisse sirpha़ isi sadhana ke jarie dekhe ja sakate haim jise prayah ‘registana ka jahaja’ kaha jata hai | rajasthana mem unta sapha़ari solahavim shatabdi se prachalana mem hai jise taba mukhya rupa se samana dhone ke lie istemala kiya jata tha, aura aba yaha manoranjana ka mahatvapurna sadhana hai | rajasthana ke thara marusthala mem chalane vale unta sapha़ari ke aja ke rupa mem prachina unta- karavam ka jadu dekha ja sakata hai | rajasthana - bhramana ka eka mahatvapurna hissa, unta sapha़ari paramparika aura ranga-birange rajasthana ki andaruni janki dikhata hai |
jaisalamera, jodhapura aura bikanera retile tilom ke lie jane jate haim para inamem se sabase lokapriya jaisalamera hai | adhikatara reta ke tile jaisalamera ke asapasa haim | saima sainda dyunsa eka uttama paryataka sthana hai jo suryasta evam suryodaya ke darshaniya sthala ke rupa mem jana jata hai | saima sainda dyunsa manava ke lie prakriti ke sarvashreshtha upaharom mem se eka hai | jaisalamera ke thara marusthala kshetra mem sthita saima sainda dyunsa sabase adhika prasiddha reta ke tilom mem se hai jo sala -bhara paryatakom ki bahuta bada़i sankhya ko akarshita karata hai | manavara marusthala bahari sahasika karanamom evam romancha ke lie sarvottama hai | paryataka apane apa ko adivasi gamvom ki yatra, marusthala ke vanya jivana ka darshana aura sthaniya shilpakarom ke shilpa ko dekhane mem vyasta rakha sakate haim | bikanera rajasthana ke uttara mem sthita hai aura samanyataya unta- pradesha ke rupa mem jana jata hai | vishva ki dasa pramukha shanadara relagada़iyom mem se eka, pailesa ऑna hvilsa, bharata ki shana kahi ja sakati hai jo yatra ki apani anupama, atulaniya, avismaraniya evam advitiya shaili ke lie jani jati hai | rajasthana ki rajasi bhumi ke darshana karane ke sarvottama madhyamom mem se eka, pailesa ऑna vhilsa bharata ke preranadayaka, atulaniya, avishvasaniya, prabhavashali evam preranatmaka kshetra se gujarati hai | bharatiya rela evam rajasthana paryataka vikasa nigama ke sanyukta prayasom se bharata ki pahali avakasha - relagada़i, pailesa ऑna vhilsa ne 1982 mem apani rajasi yatra arambha ki thi| aja yaha bhavya pailesa ऑna vhilsa relagada़i rajasi rajasthana ka anivarya hissa aura bharata ka gaurava ho gai hai jo rajasthana ke kuchha sarvottama paryataka - sthalom ke darshana karati hai | yaha rajasi yatra mai se agasta ke bicha ke mahinom ko chhoda़kara varsha - bhara hoti hai | vishva ki sarvottama romanchaka yatraom mem se eka pailesa ऑna vhilsa yatra ke daurana vishva stara ki sevaem evam suvidhaem pradana karati hai yaha rajasi yatra dilli se arambha hoti hai aura jayapura - jaisalamera - jodhapura - savai madhopura- chittauda़gadha़ - udayapura - bharatapura - agara hote hue dilli mem akara samapta hoti hai | pushkara mela pratyeka varsha hindu maha kartika (aktubara - navambara) ki shubha purnima ke avasara para lagata hai | rajasthana - bhavya, prakritika aura vastushilpiya ashcharyom se vibhushita pradesha | alabarta hala sangrahalaya jayapura, rajya ka sabase purana sangrahalaya mana jata hai |
paryatana vibhaga ne paryatana sankula ke nirmana ke lie kai nae sthanom ko nirdeshita kiya hai | hariyana paryatana ne hariyana ke sabhi kshetrom mem hotalom, motalom aura bhojanalayom ka prabandha kiya hai | panipata yamuna nadi ke tata para sthita hai aura yaham panipata ke tina aitihasika yuddha lada़e gae the | panipata eka audyogika shahara hai aura hathakaragha utpadom ke lie jana jata hai | jaham taka pata laga hai ‘danavira karna ka shahara’ karanala, divarom se ghira hua shahara raha hai aura shayada kisi samaya mem eka durga raha hoga| karanala jutom, krishi anusandhana sansthanom aura basamati chavala ke lie prasiddha hai|
panchakula jile mem uposhnakatibandhiya mahadvipiya manasuni jalavayu hai jaham hamem ritu avartana, garama grishmaritu, thandi shitaritu, aniyamita varsha aura tapamana mem atyadhika parivartana milata hai|
paryatana karane ke lie atyanta manorama sthalom mem se kuchha haim, kinnaura jile mem sangala ghati aura kalpa, shimala jile mem naladehara, narakanda aura sarahana, kullu jile mem manali aura manikarana, kangada़a jile mem dharmashala tatha lahaula aura spiti mem tabo | palamapura apane krishi mahavidyalaya aura chaya baganom ke lie prasiddha hai | himachala pradesha vanya-jivana premiyom, pakshi prekshakom tatha sahasika paryatakom ke saira karane ke lie bahuta achchhi jagaha hai | shimala paryatakom mem eka lokapriya shahara hai | shimala ki saira karane ke lie sala ke sarvottama mahine marcha - juna (vasanta), sitambara - aktubara (sharada) aura disambara - janavari (shitakalina himapata ka anubhava lene ke lie) haim | himachchhadita himalaya shrinkhala shimala se dikhai pada़ti hai | rajasthana apane paryatakom ke lie vividha prakara ke bhojana prastuta karata hai- yaha vividhata isaki sanskriti, isaki paitrika sanrachana, isaki bhaugolika sthiti aura isaki jalavayu jaisi vividha hai | da raॉyala kairija़ma ऑpha rajasthana tura (rajasi akarshana ki rajasthana yatra) ise sansara ke ati vanchhita paryataka gantavya karara deta hai | bharata ki prakritika sampada usaki sanskritika sampada ki taraha sanriddha aura vividha hai | yaham ke vanya-jiva abhayaranya aura rashtriya udyana apako vahi sara dete haim | raॉyala bangala taigara isa sanriddha kshetra mem basa hai, (jo) raॉyala taigara kaita ka ekamatra nivasa-sthana (hai)| bangala taigara sabase teja़ dauda़ne vala janavara hai | agara apa raॉyala bangala taigara mem dilachaspi rakhate haim tatha use usake svabhavika parivesha mem dekhana chahate haim, (to) ina bagha rakshita sthanom ki yatra avashya anandadayaka hogi | karbeta ka karbeta neshanala parka baghom aura satha hi unake shikara ke lie bhi ekaashraya-sthala hai, jinamem chara prakara ke hirana, jangali suara aura kuchha kama prasiddha janavara sammilita haim | svarna mandira sikkhom ke pavitra nagara anritasara mem sthita hai, jo kabhi ghana jangala tha aura gurunanaka ka nivasa-sthala tha | maidanom ke upara sthita dharmashala ghane chida़ ke peda़om aura devadara ke jangalom se ghiri hai| bahuta si jala dharaom ke satha pasa ki hima-rekha aura shitala svastha vatavarana parivesha ko bahuta hi lubhavana banata hai| kabhi shasana ka eka mahatvapurna kendra mana jane vala chanda vansha ka rajadhani shahara kangada़a, gaurava ki eka gatha kahata hai, jo itihasa mem kho gaya hai| ninna himalaya ki manorama ghatiyom mem se eka, utkrishta dhauladhara pahadiyom dvara parirakshita yaha ghati hari-bhari aura sanriddha hai | sikkima ke parvatiya kshetra mem hare-bhare pahada़i rastom, stupom, mathom aura mandirom ke satha-satha romancha ki prachurata hai | sanriddha sanskriti aura parampara ka parichaya prapta kijie jo ki apane asimita anandaprada sanskarom evam utsavom ke karana purnatah vilakshana evam mohaka hai | sikkima eka sukhadayi yatra pradana karata hai – romancha aura khoja ka eka paryatana | bahuta lambe samaya se manava apane hridaya mem eka chida़iya ki bhanti vishala nile akasha mem uda़ne ki gahari chaha lie hue hai, kuchha hada taka hama manava khule akasha mem uda़ne ke lie havai - khela ke romancha ko apanate hue apani isa pyasa ko buja pane mem sakshama haim | 26,000 phuta ki unchai para barpha se dhaki chotiyom ke shikhara vala himalaya vishva ki sarvashreshtha parvata shrrinkhala hai|
bharata mem paidala - yatra eka avismaraniya anubhava hai kyom ki yaha apako prakriti dvara nirmita ajnata rastom ka anubhava hi nahim deti,(balki) paidala - yatra paryataka ko desha aura isake logom ke satha sidhe samparka mem lati hai aura apako parvatiya vatavarana ko samajane mem sahayata karati hai | himalaya vishva ki sabase taruna parvata shrrinkhalaom mem se eka hai aura vishva ke sarvashreshtha parvatarohana kshetrom mem se gina jata hai| bharatiya himalaya shrrinkhala nissandeha vishva ki sabase bhavya aura prabhavashali parvata shrrinkhalaom mem se eka hai| thara registana mem sahasika yatra aisi yatra hai jise apa apane pure jivana mem nahim bhula paenge| paryataka usi registani anubhava ka ananda jaisalamera, rajasthana mem saima sainda dyunsa mem bhi le sakate haim jaham paryatakom ke manoranjana ke lie rajasthana paryatana vibhaga dvara sayankala mem sanskritika pradarshanom ka ayojana kiya jata hai. gira rashtriya udyana sagauna, gulamohara, khaira aura baragada vrikshom ka eka mila-jula patajada़i vana hai| satha hi gira rashtriya udyana bharata ke kisi bhi udyana ki tulana mem chite ki sabase adhika abadi vala avasa hai| bharata mem bahuta se bagha arakshita kshetra haim, jo isa khunkhara janavara ko surakshita rakha rahe haim, lekina jitani bahutayata aura niyamitata se inhem kanha rashtriya udyana mem dekha ja sakata hai vaisa aura kahim nahim | vishva ke sarvottama pakshi udyanom mem se eka, bharatapura pakshi abhayavana (kevaladeva ghana neshanala parka) eka arakshita kshetra hai jo pashu-varga ko bhi sanrakshana deta hai| udyana dekhane vale adhikansha paryatakom ka mukhya akarshana bahuta se pravasi pakshi hote haim jo saiberiya aura madhya eshiya jaise sudura pradeshom se ate haim aura apane prajanana sthanom ko lautane se pahale ve apani thanda bharatapura mem bitate haim | bharatapura abhayavana ke bada bharatapura shasakiya sangrahalaya mukhya dhyana khinchane valom mem se eka hai, jo bharatapura ke bhutakalina rajasi vaibhava ki jalaki pradana karata hai | sariska udyana tendue, jangali kutte, vana billi, lakada़bagghe, bheda़iye aura bagha ke satha-satha bahuta se mansahari pashuom ka ghara hai| sariska risasa bandarom ki adhika janasankhya ke lie suprasiddha hai, jo ki talavriksha ke asa-pasa adhika sankhya mem paye jate hai|
ranathambhaura rashtriya udyana projekta taigara rija़rvsa ऑpha da varlda (vishva ki bagha sanrakshana pariyojana) ke antargata sammilita hai aura apani bagha janasankhya ke lie adhika lokapriya hai| bagha ko usake prakritika avasa mem dekhana pratyeka vanyajivana samarthaka ki sabase vanchhaniya kalpana hai, jo ki ranathambhaura ke vana mem purna ho sakati hai| ranathambhaura vanyajivana udyana vishva ke kuchha una darshaniya sthanom mem se hai jaham baghom ka varchasva hai| ranathambhaura vanyajivana udyana daladali magaramachchha aura ubhayachara pashuom ke satha-satha chida़iyom ki 272 jatiyom aura sarisripom ka eka avasa hai| ranathambhaura vana ke chhupe hue khajane ki khoja ke lie jipa yatra bhi upalabdha karata hai, kyonki yaha raॉyala taigara ki khoja ke lie sarvottama sadhana hai| vanyajivana rashtriya udyana mem anya akarshana ranathambhaura kile ke khandahara aura jogi mahala hai jo ki 10vim shatabdi ko dinankita karate hai| aba vana vishrama griha mem parivartita jogi mahala bharata mem dusare sabase bada़e baragada ke vriksha ke lie prasiddha hai| taja nissandeha vishva ke sabase bhavya bhavanom mem se eka hai| apani sthapatya bhavyata aura kalatmaka sundarata ke lie prasiddha taja manava ki sabase gauravashali rachanaom mem gina jata hai aura vishva ke shreshtha ashcharyom ki suchi mem nirapavada rupa se sammilita hai| bharatiya pratishthita kavi taigora dvara " shashvatata ke chehare para ansu" ke rupa mem varnita, tajamahala nissandeha mugala vastukala ki parakashtha hai aura nitanta saralata se vishva ke ati ashcharyajanaka bhavanom mem se eka hai| agara apani sthapatya bhavyata ki sanriddhi, baja़arom aura abhushanom ke satha vishva ke ati utkrishta shaharom mem se hai| taja mahotsava ke samaya shilpagrama shahara mem pure vishva ke paryataka ekatra hote haim aura isaka bharapura ananda lete haim| agara ane vale paryataka taja ko apani yadom mem banae rakhane ke lie apane satha isaka eka namuna avashya lem jaenge| agara mem chamada़e ke thailom, kasidakari vale jutom ki kharidadari ka bhi bada़a akarshana hai aura petha (mithai) to nishchita rupa se agara ki eka visheshata hai| yadyapi sudura mem sthita hai, phira bhi khajuraho mandira sankula sthala videshi aura bharatiya donom paryatakom ke lie sarvadhika lokapriya sthanom mem se eka hai| khajuraho ke mandira apani murti kala ke satha paryataka ka dhyana banae rakhate haim, jo ki bahuta utkarsha aura jatila hai, jise aba koi bhi hubahu banane ka sapana bhi nahim dekha sakata| nishpadana mem shreshtha aura bhavabhivyakti mem utkrishta khajuraho mandira naritva ko samarpana haim| kalakara ki srijanatmaka pravrittiyom ne jivana ke vibhinna pahaluom aura bhavom ko patthara mem sundarata se utara hai|'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(25)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
7
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
4
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
1
1


In [145]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =['\'', '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']

        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        for i in data['Freq']:
            print(i)
        #for i in data['Pairs']:
         #   print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''muhammada hanipha rajagiri ke nirikshaka the| tajamahala ke dhanche ki kalpana karane vale ustada ahamada lauhori the| tajamahala ke lie nirmana samagri pure bharata emva madhya eshiya se lai gai thi aura use nirmana sthala para lane ke lie 1000 hathiyom ke junda ki jarurata pada़i| tajamahala ke lie lala retile patthara phateha pura sikari se, suryakanta panjaba se, haritashma aura sphatika china se, phiroji patthara tibbata se, nila bahumulya ratna aura nilama shrilanka se, koyala aura kaॉrneliyana araba aura hire panna se mangaye jate the| kula milakara 18 prakara ke virala, ratna aura bahumulya ratna tajamahala ke jada़au kama ke lie upayoga mem lie gaye the| tajamahala ke lie mukhya nirmana bhavana samagri, sapheda sangamaramara, rajasthana ke nagaura jile mem makarana ki khanom se laya gaya| tajamahala ki shilpakala mem eka akarshaka nahara hai, jo ki udyana ko chara barabara bhagom mem katati hai aura sampurna nirmana ki dosharahita sudaulata ko badha़ati hai| charabaga mem nahara aura jalashaya taja ka eka bhavya pratibimba deta hai jo ki svarga ki kalpanika akriti ko aura dridha़ karata hai| vastukarom, jinhonne tajamahala ka nirmana kiya, ne ishvariya prerana prapta karane ke lie, tajamahala ke ulte pratibimba ko paida karane ke lakshya se udyanom mem naharom aura jalakalom ko banaya| tajamahala ki purnata ke bada charabaga ke andara hara baga ko 16 phulom ke galichom mem vibhajita kiya gaya jo ki kula 64 ho gayem| aisa kaha jata hai ki pratyaika phulabaga ko 400 paudhom se sthapita kiya gaya| tajamahala mem peda़om ko sampurna yojana ke mela ke anusara savadhani purvaka lagaya gaya| ye peda़ sabase virala pakshiyom mem se kuchha ke ghara haim,jinamem se sabhi taja ke ashcharyajanaka vatavarana mem badhotari karate haim| bagichem mem ada़i tirachhi bahati hui jaladhara vibhinna jatiyom ki rangina machhalaliyom se bhari rahati hai| isake jalakala, udyana ke rakharakhava para vishesha dhyana diya jata tha, kyonki shahi parivara ke sadasya aksara vaham jaya karate the evam usake pasa ke atithi griha (mehamana khana) mem ruka karate the| agara shahara taja mahala ke lie jagata prasidhda hai jo ki mugala badashaha shahajaham dvara apani priya patni ki yada mem banaya gaya tha|
yadyapi, yaha agara kile ke lie bhi prasighda hai, jo mugala vastushilpiya parampara ka vastavika kosha haim| isa bhavya kile samuha mem, vibhinna bhavana ,aneka sanskritiyom ka milana prastuta karate haim, jo mugala kala ki nishani the| tajamahala ki banavata mugala shaili ki vastukala ka anusarana karati hai jo bharata mem madhyakala ke daurana phali phuli| vastukala ki mugala shaili ko vastukala ki bharatiya-mugala shaili se bhi ullekhita kiya jata haim| yaha shobhayamana smaraka charabaga ya chara bagichom ke charom ora sthapita jalamarga se alaga hai jo ki pharasi shaili ka pratibimba hai| ajanta ki guphaem aurangabada shahara se 107 ki.mi. taka phaili hui hai| ajanta ki chattana katakara banai gai guphaem parvata ke bicha ke eka vistrita marga mem eka vishalakaya ghoda़e ki nala ke rupa mem sthita hai| prachinatama baudhda shilpakala, gupha ke chitrom aura murtiyom mem sem kuchha ke sarvottama udaharanom mem se eka ye guphaem chaitya pahada़ ya pavitra sthalom jo ki bhagavana budhda ko samarpita hai aura vihara ya mathom jom ki baudhda sanyasiyom dvara dhyana emva baudhda upadeshom ke adhyayana kelie kama mem lie jate the ko sammilita kie haim| guphaom ki divarom aura bhitari chhata para jo chitrakariyam abhushita haim, budhda ke jivana tatha baudhda devatvom ki divyataom ki ghatanaom ko chitrita karati haim| jyada ruchikara chitrom mem se jataka kathaem haim, jo ki budhda ke purva janmom, jaise ki bodhisatva, eka sidhda purusha jo ki budhda banane ke lie lakshita tha, se sambadhita vibhinna kahaniyom ko chitrita karati haim| 700 salom se tiki hui ajanta ki guphaem prayah ekaeka parityakta maluma hoti haim| usa chattana se niche girata hua eka darshaniya jarana hai, jo tala para eka prakritika talaba ko bharata hai jise saptakunda kaha jata hai| 
ajanta eka virasata ki taraha nirvachita ki gai hai| yaha eka kalatmaka dharohara ki taraha bachani hai jo ki ane vali pushtom ko prerita aura unaki jindagiyom ko sushobhita karana jari rakhengi| ajanta ki guphaom ko do alaga charanom mem vargikrita kiya ja sakata hai, pahale vala hinayana jisamem budhda sirpha kuchha pratikom ke rupa mem hi puje jate haim| ailiphenta ka itihasa samaya ki puratana dhundha hai|
sira ke upara upasthita bagule achala rupa se mandarate rahate haim aura jaba kadachita kala kala bahate jala se upara uchhalati hui machhaliyom para japatate haim to vaha eka suhavana sava ghante ka eliphenta ka saphara hota hai| kabhi puri nama se jana jane vala, bada mem ghadhapuri, yaha dvipa shaktishali tatavarti rajya ki garvita rajadhani thi aura chhathi shatabdi mem khoji gayi, shiva ki prashansa mem mahana pavitra gupha, prabala rajavansha mem joda़i gai| kai shatabdiyom bada purtagaliyom ne dvipa para kabja kara liya|
ajanta ki shreshta mathom mem se eka ajanta ki sampurna guphaem haim| ajanta ki sampurna guphame shravasti ke chamatkara ka pratinidhitva karati haim, jaba budhda ne hajarom rupom mem svayam ko prakata kiya tha| ajanta ki sampurna guphaom ke pasa eka madhya daravaja hai jo ki sanrakshakom, uda़te hue chitrom, peda़om ko pakada़e hui kanyaom se aura harom ke satha budhda evam ganom ya baunom ki pratimaom se bhi saja hua haim| mumbai bharata ka antarrashtriya pravesha dvara haim| khajuraho ke mandira madhyakalina bharatiya vastukala ke utkrishta udaharana hai| yatra ki avashyakata ke anusara kamarom ka arakshana purva mem bhali bhanti kiya ja sakata hai| nai dilli ki pratishtita dharohara ke bare mem janane ka sabase uttama tarika hai usake smaraka chinhom ko janana| bharata apane badha़te hue deshatana aura paryatana vibhaga ko unnata banane ke lie aneka bhavya hotalom aura sairagahom ki vyavastha karata hai| phatehapura sikari lala retile patthara se nirmita eka shahara hai| sampurna sansara mem shanadara tajamahala sabase adhika prachalita samadhi hua karata hai| vishvabhara se paryataka apravahi jala ko dekhane ke lie ate haim| bharata sarakara ne uttara-purvi pranta mem paryatana ko badha़ava dene ke lie videshi paryatakom ko nagalainda ki yatra karane ki svikriti dene ka nishchaya kiya| varna aura ranga ki prachina yugina kala akarshana ka kendra hai| maulika mahala raja mana sinha dvara banaya gaya tha aura bada mem savai jaya sinha ke dvara parivardhana kiya gaya| bharata eka vishala desha hai tatha isaki bhautika simaem dura taka chauda़ai mem phaili hui haim| kerala anupama bhaugolika lakshanom ka dhani hai jinhone use eshiya ke sabase jyada vanchhita paryatana sthalom mem se eka bana diya hai| bharata ke kai anupama pradeshom taka jipom dvara bhali prakara pahuncha ja sakata hai| 
kuchha chijem aisi haim jinhem lakshadvipa mem khojana sachamucha mushkila hai| yaha bada़a kamara, jisamem shikara ke vijaya chinha ka sangraha hai, aba eka sangrahalaya mem badala diya gaya hai| kerala aura tamilanadu ki janasankhya mem se kariba eka pratishata adivasi hai| jabaki andhrapradesha evam karnataka mem kariba chhah pratishata adivasi samuhom ke sadasya haim| maharaja gangasinha dvara apane pita ki yada mem nirmita, lalagadha mahala bikanera shahara se 3 ki.mi. uttara mem sthita hai| bharata, ananta bhinnata ka sthana, apane prachina aura jatila sanskriti ke satha akarshaka hai| kerala ka saundarya matra likha ya samajaya nahim ja sakata, use anubhava karana hota hai| kerala apani pashchima sima araba sagara ke satha bantata hai| 
tajamahala ke bagom ke pasa mugala smaraka, jo ki agara ka lala kila jana jata hai, khada़a hai| 
hava mahala jayapura ka sabase adhika chiraparichita smaraka hai| siti pailesa shahara ke madhya mem sthita hai| yaha prachina nagara kshetra ka lagabhaga eka satavam kshetra gherata hai| gaitura jayapura ke rajaom ka antima vishrama sthala hai aura jayapura amera sada़ka kuchha hi dura sthita hai| jayapura bharatiya eyara lainsa ki niyamita uda़anom dvara dilli, bambai, ahamadabada se achchhi taraha juda़a hua hai| kanuna aura vyavastha ki drishti se bharata mem sabase surakshita rajya rajasthana hai| yaha apane paryatakom ko eka surakshita aura nihshanka paryavarana pradana karata hai| jayapura dilli se 262 ki.mi. dura hai aura sada़ka, rela evam vayumargom dvara sare desha se bhali-bhanti juda़a hai| bharata ke pasa hara sahasi karya chahane vale ke lie kuchha hai| pashchima tata mumbai, gova aura puna ke pasa jilom mem nauka vihara pradana karata hai| 
jala krida़a ke lie sarvottama mahine aktubara aura marcha haim| bharata vanya jivom ki vibhinnataom ki drishti se bharapura hai/ bharata mem vanya jivom ki atyadhika vividhata pai jati hai| sadharana nagarikom ke lie eka paryataka shabdakosha likha hua hai| bharata sansara ke eka sau gaindom ki samasta janasankhya mem assi pratishata ka thikana hai| rajasi bambai ka nirmana 20vim shatabdi mem nirantara achchhi taraha se chala| jala vibhaga, julate baga evam jilem bhi isi samaya banai gai thim| nauka dauda़ jo ki kerala ke sthira pani mem rakhi jati hai dusari ghatana hai jo ki kabhi bhi chhoda़ni nahi chahiye| mausama ke bare mem kuchha tathya haim ki kerala mem baki bharata ke samana sukha samaya nahim hota hai| jaisalamera ka kila duniya ka sabase purana kila hai| rajasi thatha ka pahala anubhava pailesa ऑna vhilsa para ho sakata hai jo ki rajasthana paryatana vibhaga dvara chalai jane vali eka paryataka relagada़i hai| ghoda़om ki yatra rajasthana ke maravada jile mem ayojita hoti hai| jayapura nagara uttara bharata ke rajasthana rajya ki rajadhani hai aura apani rangina sanskriti, kilom, ma/halom aura jilom ke lie prasidhda hai| bharata, chirakala se sanskritika sthana kaha jane vala aura bahari sthana ki taraha vistrita romanchaka vikalpom ke satha jana jata hai| 14vim shatabdi ka pahada़i kila, kesaroli bharata ka sabase purani virasata mem mila sthala hai jaham apa ruka sakate haim|yaha 500 sala purana hai jo ki samaya mem sthira khada़a hai| 
prinsa elbarta, inglainda ki maharani viktoriya ke pati jinhone 1883 isvi mem bharata bhramana kiya, ke svagata mem jayapura ko gulabi ranga se ranga gaya| ye nritya samanyataya vishesha rupa se lagae gae manchom para mandira ke utsavom ke sambandha mem kiya jata hai| pailesa ऑna vhilsa rajaom ke yogya yatra hai| vyavasthita aramadayaka basem evam vishesha relagada़iyam jaise ki shatabdi eksapresa aura intarasiti eksapresa chaubisa ghante upalabdha hai| parishkrita guphaem ajanta mem uttama mathom mem se eka haim| 
ajanta ki guphaem aurangabada shahara se 107 ki.mi. taka phaili hui haim| ajanta ki chattana katakara banai gai guphaem parvata ke bicha ke eka vistrita marga mem eka vishalakaya ghoda़e ki nala ke rupa mem sthita haim| ailiphenta ka itihasa samaya ki sapheda dhundha hai| aurangabada, nikatatama rela sthala, mumbai, dilli, agara, aura bhopala se sidhe juda़a hua hai| aurangabada sare bada़e shaharom aura nagarom se achchhi sada़kom dvara juda़a hai| maharashtra rajya sada़ka parivahana nigama aurangabada se mumbai ke lie sadharana aura shanadara basem chalati hai| khudai ki dusari avastha ki adhuri mathem ajanta ki adhuri guphaem haim| 
majara para upasthita shanadara gumbada sanrata shahajaham dvara banaya gaya, jinhonne sala 1638 mem jama masjida ka bhi nirmana karaya| daragaha ka mukhya dvara, nijama daravaja, 70 phita uncha hai| yaha haidarabada ke nijama dvara banaya gaya tha| aitihasika bulanda daravaja jo ki 85 phita uncha hai, mandu ke sultana gayasuddina dvara banaya gaya tha|mahaphila khana ya mukhya bada़a kamara daragaha ki vaha jagaha hai jaham salabhara kavvaliyam gai jati haim| daragaha ke uttara mem eka langara khana hai jaham khana pakaya jata hai aura garibom aura jaruratamandom mem dina mem do bara banta jata hai| ursa hara sala rajaba ki pahali tarikha se navim taka manaya jata hai| ajamera jila jo ki registana ke kinare sthita hai aura tina tarapha se pahada़om se ghiri hai, ajamera se naga paharthe sneka pahada़ dvara prithaka hai| naga paharthe sneka pahada़ para panchakunda aura santa agastya ki gupha sthita hai| aisa mana jata hai ki chauthi shatabdi ke kavi kalidasa ne apani shreshta rachana "abhijnana shakuntalama" natika ke lie paitrika sarpa vana mem naga paharthe ko chuna|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

4
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [146]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =['\'', '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']

        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        #for i in data['Freq']:
         #   print(i)
        for i in data['Pairs']:
            print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''muhammada hanipha rajagiri ke nirikshaka the| tajamahala ke dhanche ki kalpana karane vale ustada ahamada lauhori the| tajamahala ke lie nirmana samagri pure bharata emva madhya eshiya se lai gai thi aura use nirmana sthala para lane ke lie 1000 hathiyom ke junda ki jarurata pada़i| tajamahala ke lie lala retile patthara phateha pura sikari se, suryakanta panjaba se, haritashma aura sphatika china se, phiroji patthara tibbata se, nila bahumulya ratna aura nilama shrilanka se, koyala aura kaॉrneliyana araba aura hire panna se mangaye jate the| kula milakara 18 prakara ke virala, ratna aura bahumulya ratna tajamahala ke jada़au kama ke lie upayoga mem lie gaye the| tajamahala ke lie mukhya nirmana bhavana samagri, sapheda sangamaramara, rajasthana ke nagaura jile mem makarana ki khanom se laya gaya| tajamahala ki shilpakala mem eka akarshaka nahara hai, jo ki udyana ko chara barabara bhagom mem katati hai aura sampurna nirmana ki dosharahita sudaulata ko badha़ati hai| charabaga mem nahara aura jalashaya taja ka eka bhavya pratibimba deta hai jo ki svarga ki kalpanika akriti ko aura dridha़ karata hai| vastukarom, jinhonne tajamahala ka nirmana kiya, ne ishvariya prerana prapta karane ke lie, tajamahala ke ulte pratibimba ko paida karane ke lakshya se udyanom mem naharom aura jalakalom ko banaya| tajamahala ki purnata ke bada charabaga ke andara hara baga ko 16 phulom ke galichom mem vibhajita kiya gaya jo ki kula 64 ho gayem| aisa kaha jata hai ki pratyaika phulabaga ko 400 paudhom se sthapita kiya gaya| tajamahala mem peda़om ko sampurna yojana ke mela ke anusara savadhani purvaka lagaya gaya| ye peda़ sabase virala pakshiyom mem se kuchha ke ghara haim,jinamem se sabhi taja ke ashcharyajanaka vatavarana mem badhotari karate haim| bagichem mem ada़i tirachhi bahati hui jaladhara vibhinna jatiyom ki rangina machhalaliyom se bhari rahati hai| isake jalakala, udyana ke rakharakhava para vishesha dhyana diya jata tha, kyonki shahi parivara ke sadasya aksara vaham jaya karate the evam usake pasa ke atithi griha (mehamana khana) mem ruka karate the| agara shahara taja mahala ke lie jagata prasidhda hai jo ki mugala badashaha shahajaham dvara apani priya patni ki yada mem banaya gaya tha|
yadyapi, yaha agara kile ke lie bhi prasighda hai, jo mugala vastushilpiya parampara ka vastavika kosha haim| isa bhavya kile samuha mem, vibhinna bhavana ,aneka sanskritiyom ka milana prastuta karate haim, jo mugala kala ki nishani the| tajamahala ki banavata mugala shaili ki vastukala ka anusarana karati hai jo bharata mem madhyakala ke daurana phali phuli| vastukala ki mugala shaili ko vastukala ki bharatiya-mugala shaili se bhi ullekhita kiya jata haim| yaha shobhayamana smaraka charabaga ya chara bagichom ke charom ora sthapita jalamarga se alaga hai jo ki pharasi shaili ka pratibimba hai| ajanta ki guphaem aurangabada shahara se 107 ki.mi. taka phaili hui hai| ajanta ki chattana katakara banai gai guphaem parvata ke bicha ke eka vistrita marga mem eka vishalakaya ghoda़e ki nala ke rupa mem sthita hai| prachinatama baudhda shilpakala, gupha ke chitrom aura murtiyom mem sem kuchha ke sarvottama udaharanom mem se eka ye guphaem chaitya pahada़ ya pavitra sthalom jo ki bhagavana budhda ko samarpita hai aura vihara ya mathom jom ki baudhda sanyasiyom dvara dhyana emva baudhda upadeshom ke adhyayana kelie kama mem lie jate the ko sammilita kie haim| guphaom ki divarom aura bhitari chhata para jo chitrakariyam abhushita haim, budhda ke jivana tatha baudhda devatvom ki divyataom ki ghatanaom ko chitrita karati haim| jyada ruchikara chitrom mem se jataka kathaem haim, jo ki budhda ke purva janmom, jaise ki bodhisatva, eka sidhda purusha jo ki budhda banane ke lie lakshita tha, se sambadhita vibhinna kahaniyom ko chitrita karati haim| 700 salom se tiki hui ajanta ki guphaem prayah ekaeka parityakta maluma hoti haim| usa chattana se niche girata hua eka darshaniya jarana hai, jo tala para eka prakritika talaba ko bharata hai jise saptakunda kaha jata hai| 
ajanta eka virasata ki taraha nirvachita ki gai hai| yaha eka kalatmaka dharohara ki taraha bachani hai jo ki ane vali pushtom ko prerita aura unaki jindagiyom ko sushobhita karana jari rakhengi| ajanta ki guphaom ko do alaga charanom mem vargikrita kiya ja sakata hai, pahale vala hinayana jisamem budhda sirpha kuchha pratikom ke rupa mem hi puje jate haim| ailiphenta ka itihasa samaya ki puratana dhundha hai|
sira ke upara upasthita bagule achala rupa se mandarate rahate haim aura jaba kadachita kala kala bahate jala se upara uchhalati hui machhaliyom para japatate haim to vaha eka suhavana sava ghante ka eliphenta ka saphara hota hai| kabhi puri nama se jana jane vala, bada mem ghadhapuri, yaha dvipa shaktishali tatavarti rajya ki garvita rajadhani thi aura chhathi shatabdi mem khoji gayi, shiva ki prashansa mem mahana pavitra gupha, prabala rajavansha mem joda़i gai| kai shatabdiyom bada purtagaliyom ne dvipa para kabja kara liya|
ajanta ki shreshta mathom mem se eka ajanta ki sampurna guphaem haim| ajanta ki sampurna guphame shravasti ke chamatkara ka pratinidhitva karati haim, jaba budhda ne hajarom rupom mem svayam ko prakata kiya tha| ajanta ki sampurna guphaom ke pasa eka madhya daravaja hai jo ki sanrakshakom, uda़te hue chitrom, peda़om ko pakada़e hui kanyaom se aura harom ke satha budhda evam ganom ya baunom ki pratimaom se bhi saja hua haim| mumbai bharata ka antarrashtriya pravesha dvara haim| khajuraho ke mandira madhyakalina bharatiya vastukala ke utkrishta udaharana hai| yatra ki avashyakata ke anusara kamarom ka arakshana purva mem bhali bhanti kiya ja sakata hai| nai dilli ki pratishtita dharohara ke bare mem janane ka sabase uttama tarika hai usake smaraka chinhom ko janana| bharata apane badha़te hue deshatana aura paryatana vibhaga ko unnata banane ke lie aneka bhavya hotalom aura sairagahom ki vyavastha karata hai| phatehapura sikari lala retile patthara se nirmita eka shahara hai| sampurna sansara mem shanadara tajamahala sabase adhika prachalita samadhi hua karata hai| vishvabhara se paryataka apravahi jala ko dekhane ke lie ate haim| bharata sarakara ne uttara-purvi pranta mem paryatana ko badha़ava dene ke lie videshi paryatakom ko nagalainda ki yatra karane ki svikriti dene ka nishchaya kiya| varna aura ranga ki prachina yugina kala akarshana ka kendra hai| maulika mahala raja mana sinha dvara banaya gaya tha aura bada mem savai jaya sinha ke dvara parivardhana kiya gaya| bharata eka vishala desha hai tatha isaki bhautika simaem dura taka chauda़ai mem phaili hui haim| kerala anupama bhaugolika lakshanom ka dhani hai jinhone use eshiya ke sabase jyada vanchhita paryatana sthalom mem se eka bana diya hai| bharata ke kai anupama pradeshom taka jipom dvara bhali prakara pahuncha ja sakata hai| 
kuchha chijem aisi haim jinhem lakshadvipa mem khojana sachamucha mushkila hai| yaha bada़a kamara, jisamem shikara ke vijaya chinha ka sangraha hai, aba eka sangrahalaya mem badala diya gaya hai| kerala aura tamilanadu ki janasankhya mem se kariba eka pratishata adivasi hai| jabaki andhrapradesha evam karnataka mem kariba chhah pratishata adivasi samuhom ke sadasya haim| maharaja gangasinha dvara apane pita ki yada mem nirmita, lalagadha mahala bikanera shahara se 3 ki.mi. uttara mem sthita hai| bharata, ananta bhinnata ka sthana, apane prachina aura jatila sanskriti ke satha akarshaka hai| kerala ka saundarya matra likha ya samajaya nahim ja sakata, use anubhava karana hota hai| kerala apani pashchima sima araba sagara ke satha bantata hai| 
tajamahala ke bagom ke pasa mugala smaraka, jo ki agara ka lala kila jana jata hai, khada़a hai| 
hava mahala jayapura ka sabase adhika chiraparichita smaraka hai| siti pailesa shahara ke madhya mem sthita hai| yaha prachina nagara kshetra ka lagabhaga eka satavam kshetra gherata hai| gaitura jayapura ke rajaom ka antima vishrama sthala hai aura jayapura amera sada़ka kuchha hi dura sthita hai| jayapura bharatiya eyara lainsa ki niyamita uda़anom dvara dilli, bambai, ahamadabada se achchhi taraha juda़a hua hai| kanuna aura vyavastha ki drishti se bharata mem sabase surakshita rajya rajasthana hai| yaha apane paryatakom ko eka surakshita aura nihshanka paryavarana pradana karata hai| jayapura dilli se 262 ki.mi. dura hai aura sada़ka, rela evam vayumargom dvara sare desha se bhali-bhanti juda़a hai| bharata ke pasa hara sahasi karya chahane vale ke lie kuchha hai| pashchima tata mumbai, gova aura puna ke pasa jilom mem nauka vihara pradana karata hai| 
jala krida़a ke lie sarvottama mahine aktubara aura marcha haim| bharata vanya jivom ki vibhinnataom ki drishti se bharapura hai/ bharata mem vanya jivom ki atyadhika vividhata pai jati hai| sadharana nagarikom ke lie eka paryataka shabdakosha likha hua hai| bharata sansara ke eka sau gaindom ki samasta janasankhya mem assi pratishata ka thikana hai| rajasi bambai ka nirmana 20vim shatabdi mem nirantara achchhi taraha se chala| jala vibhaga, julate baga evam jilem bhi isi samaya banai gai thim| nauka dauda़ jo ki kerala ke sthira pani mem rakhi jati hai dusari ghatana hai jo ki kabhi bhi chhoda़ni nahi chahiye| mausama ke bare mem kuchha tathya haim ki kerala mem baki bharata ke samana sukha samaya nahim hota hai| jaisalamera ka kila duniya ka sabase purana kila hai| rajasi thatha ka pahala anubhava pailesa ऑna vhilsa para ho sakata hai jo ki rajasthana paryatana vibhaga dvara chalai jane vali eka paryataka relagada़i hai| ghoda़om ki yatra rajasthana ke maravada jile mem ayojita hoti hai| jayapura nagara uttara bharata ke rajasthana rajya ki rajadhani hai aura apani rangina sanskriti, kilom, ma/halom aura jilom ke lie prasidhda hai| bharata, chirakala se sanskritika sthana kaha jane vala aura bahari sthana ki taraha vistrita romanchaka vikalpom ke satha jana jata hai| 14vim shatabdi ka pahada़i kila, kesaroli bharata ka sabase purani virasata mem mila sthala hai jaham apa ruka sakate haim|yaha 500 sala purana hai jo ki samaya mem sthira khada़a hai| 
prinsa elbarta, inglainda ki maharani viktoriya ke pati jinhone 1883 isvi mem bharata bhramana kiya, ke svagata mem jayapura ko gulabi ranga se ranga gaya| ye nritya samanyataya vishesha rupa se lagae gae manchom para mandira ke utsavom ke sambandha mem kiya jata hai| pailesa ऑna vhilsa rajaom ke yogya yatra hai| vyavasthita aramadayaka basem evam vishesha relagada़iyam jaise ki shatabdi eksapresa aura intarasiti eksapresa chaubisa ghante upalabdha hai| parishkrita guphaem ajanta mem uttama mathom mem se eka haim| 
ajanta ki guphaem aurangabada shahara se 107 ki.mi. taka phaili hui haim| ajanta ki chattana katakara banai gai guphaem parvata ke bicha ke eka vistrita marga mem eka vishalakaya ghoda़e ki nala ke rupa mem sthita haim| ailiphenta ka itihasa samaya ki sapheda dhundha hai| aurangabada, nikatatama rela sthala, mumbai, dilli, agara, aura bhopala se sidhe juda़a hua hai| aurangabada sare bada़e shaharom aura nagarom se achchhi sada़kom dvara juda़a hai| maharashtra rajya sada़ka parivahana nigama aurangabada se mumbai ke lie sadharana aura shanadara basem chalati hai| khudai ki dusari avastha ki adhuri mathem ajanta ki adhuri guphaem haim| 
majara para upasthita shanadara gumbada sanrata shahajaham dvara banaya gaya, jinhonne sala 1638 mem jama masjida ka bhi nirmana karaya| daragaha ka mukhya dvara, nijama daravaja, 70 phita uncha hai| yaha haidarabada ke nijama dvara banaya gaya tha| aitihasika bulanda daravaja jo ki 85 phita uncha hai, mandu ke sultana gayasuddina dvara banaya gaya tha|mahaphila khana ya mukhya bada़a kamara daragaha ki vaha jagaha hai jaham salabhara kavvaliyam gai jati haim| daragaha ke uttara mem eka langara khana hai jaham khana pakaya jata hai aura garibom aura jaruratamandom mem dina mem do bara banta jata hai| ursa hara sala rajaba ki pahali tarikha se navim taka manaya jata hai| ajamera jila jo ki registana ke kinare sthita hai aura tina tarapha se pahada़om se ghiri hai, ajamera se naga paharthe sneka pahada़ dvara prithaka hai| naga paharthe sneka pahada़ para panchakunda aura santa agastya ki gupha sthita hai| aisa mana jata hai ki chauthi shatabdi ke kavi kalidasa ne apani shreshta rachana "abhijnana shakuntalama" natika ke lie paitrika sarpa vana mem naga paharthe ko chuna|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

('the|', 'tajamahala')
('ajanta', 'sampurna')
('banai', 'gai')
('ऑna', 'vhilsa')
('ailiphenta', 'itihasa')
('chattana', 'katakara')
('ajanta', 'chattana')
('phaili', 'ajanta')
('107', 'ki.mi')
('shahara', '107')
('bahumulya', 'ratna')
('guphaem', 'aurangabada')
('bada़a', 'kamara')
('ajanta', 'guphaem')
('itihasa', 'samaya')
('chitrita', 'karati')
('paryatana', 'vibhaga')
('mugala', 'shaili')
('bhali', 'bhanti')
('jana', 'jata')
('banaya', 'tha|')
('gaya|', 'tajamahala')
('guphaem', 'ajanta')
('phita', 'uncha')
('bharata', 'vanya')
('vanya', 'jivom')
('katakara', 'banai')
('aurangabada', 'shahara')
('gai', 'guphaem')
('jane', 'vala')
('naga', 'paharthe')
('paharthe', 'sneka')
('retile', 'patthara')
('sneka', 'pahada़')
('parvata', 'bicha')
('bicha', 'vistrita')
('vistrita', 'marga')
('ghoda़e', 'nala')
('guphaem', 'parvata')
('pasa', 'jilom')
('paryavarana', 'pradana')
('pradana', 'jayapura')
('puna', 'pasa')
('nauka', 'vihara')
('nihshanka', 'paryavarana')
('mahine', 'aktubara')
('pra

('rupa', 'sthita')
('sthita', 'ailiphenta')
('sapheda', 'dhundha')
('viktoriya', 'pati')
('dhundha', 'aurangabada')
('aurangabada', 'nikatatama')
('nikatatama', 'rela')
('rela', 'sthala')
('sthala', 'mumbai')
('mumbai', 'dilli')
('dilli', 'agara')
('agara', 'bhopala')
('sidhe', 'juda़a')
('eksapresa', 'intarasiti')
('shatabdi', 'eksapresa')
('basem', 'vishesha')
('aramadayaka', 'basem')
('pati', 'jinhone')
('jinhone', '1883')
('1883', 'isvi')
('isvi', 'bharata')
('bharata', 'bhramana')
('bhramana', 'svagata')
('svagata', 'jayapura')
('jayapura', 'gulabi')
('gulabi', 'ranga')
('ranga', 'gaya|')
('nritya', 'samanyataya')
('samanyataya', 'vishesha')
('vishesha', 'rupa')
('lagae', 'gae')
('gae', 'manchom')
('manchom', 'mandira')
('vhilsa', 'rajaom')
('yogya', 'yatra')
('vyavasthita', 'aramadayaka')
('kendra', 'maulika')
('bharata', 'sarakara')
('prachina', 'yugina')
('mehamana', 'khana')
('jata', 'kyonki')
('kyonki', 'shahi')
('shahi', 'parivara')
('sadasya', 'aksara')
('aksara', 'jaya')
(

('adhyayana', 'kelie')
('kelie', 'kama')
('jate', 'sammilita')
('sammilita', 'kie')
('kie', 'guphaom')
('divarom', 'bhitari')
('bhitari', 'chhata')
('chitrakariyam', 'abhushita')
('budhda', 'jivana')
('jivana', 'tatha')
('prayah', 'ekaeka')
('parityakta', 'maluma')
('jana', 'jane')
('bahate', 'jala')
('upara', 'upasthita')
('upasthita', 'bagule')
('bagule', 'achala')
('achala', 'rupa')
('rupa', 'mandarate')
('mandarate', 'rahate')
('rahate', 'kadachita')
('kala', 'kala')
('kala', 'bahate')
('upara', 'uchhalati')
('maluma', 'chattana')
('uchhalati', 'machhaliyom')
('machhaliyom', 'japatate')
('japatate', 'suhavana')
('suhavana', 'sava')
('sava', 'ghante')
('eliphenta', 'saphara')
('saphara', 'kabhi')
('kabhi', 'puri')
('puri', 'nama')
('puratana', 'dhundha')
('jate', 'ailiphenta')
('puje', 'jate')
('sirpha', 'pratikom')
('darshaniya', 'jarana')
('prakritika', 'talaba')
('bharata', 'saptakunda')
('saptakunda', 'jata')
('jata', 'ajanta')
('kalatmaka', 'dharohara')
('ane', 'vali')
('vali',